In [1]:
import pyspark
import numpy as np
import pandas as pd
import os
from os import walk
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql.functions import regexp_replace
from pyspark.sql.functions import array_contains
from pyspark.sql.functions import *
import time
from pyspark.sql.functions import date_format
from datetime import date
import datetime
from datetime import datetime
import re
import warnings
warnings.filterwarnings('ignore')

date1 = input('Enter a date in YYYY-MM-DD format ')                                     ## TO BE CHANGED DAILY AS PER UPDATION DATE
year , month , day = map(int,date1.split('-'))
day , month = f"{day:02d}",f"{month:02d}"
tablename="r"+str(day)+str(month)+str(year)
print(tablename)

st=time.time()

def banknifty_data():
    spark = SparkSession.builder.config("spark.jars", "C:\\Users\\admin\\Downloads\\postgresql-42.5.0.jar") \
    .master("local").appName("PySpark_Postgres_test").getOrCreate()
    
    df = spark.read.format("jdbc").option("url", "jdbc:postgresql://swandatabase.cfehmk2wtejq.ap-south-1.rds.amazonaws.com/RawDataBase").option("user","postgres").option("password","swancap123")\
        .option("driver", "org.postgresql.Driver").option("dbtable", tablename)\
        .option("user", "postgres").option("password", "swancap123").load()

    ## GETTING ONLY TIME IN TIME COLUMN
    q = df.withColumn('time',date_format('time', 'HH:mm:ss'))
    bndata = q.filter(q.ticker.contains('BANKNIFTY') & ((q.ticker.endswith('E.NFO'))| (q.ticker.endswith('E'))))
    ## REPLACING .NFO IN ticker
    bndata = bndata.withColumn('ticker',regexp_replace('ticker','.NFO',''))

    ## CONVERTING PYSPARK DATAFRAME TO PANDAS DATAFRAME
    bndata = bndata.toPandas()
    return bndata

def banknifty_monthly():
    
    ## CHECKING IF PATH DOES NOT EXIST, THEN CREATE PATH
    if not os.path.exists(r"C:\Users\admin\Desktop\Pyspark_Contracts\BankNifty\Monthly_data\\"):
        os.makedirs(r"C:\Users\admin\Desktop\Pyspark_Contracts\BankNifty\Monthly_data\\")
        
    if not os.path.exists(r"C:\users\admin\desktop\Pyspark\BankNifty\Monthly\\"):
        os.makedirs(r"C:\users\admin\desktop\Pyspark\BankNifty\Monthly\\")
    
    folpath = r"C:\Users\admin\Desktop\Pyspark_Contracts\BankNifty\Monthly_data\\"
    sym = 'BANKNIFTY'
    start_time = datetime.strptime('09:15:00', '%H:%M:%S').time()
    end_time = datetime.strptime('15:30:00', '%H:%M:%S').time()
    
    def add(stri):
        obj = datetime.strptime(stri, "%b")
        month_number = obj.month
        return month_number

    ## READING THE EXPIRY SHEET
    exp_file_path = r"C:\Users\admin\Downloads\MonthlyExpiry.csv"
    exp_df = pd.read_csv(exp_file_path,parse_dates=["curr_exp_date","curr_date"],dayfirst=True).dropna()
    ## CONVERTING TO PANDAS DATAFRAME
    exp_df.rename({'curr_date': 'New_date'}, axis=1, inplace=True)
    exp_df['New_date'] = pd.to_datetime(exp_df['New_date'],dayfirst=True)
    exp_date = pd.read_excel(r'C:\users\admin\desktop\Expiry_DT.xlsx')
    
    ## CALLING FUNCTION BANKNIFTY_DATA TO GENERATE ONLY BANKNIFTY TICKERS
    ddf = banknifty_data()
    ddf = ddf.loc[:, ~ddf.columns.str.contains('^Unnamed')]
    ddf['date'] = pd.to_datetime(ddf['date'],dayfirst=True)
    ddf['Optiontype'] = ddf['ticker'].str[-2:]
    ddf['Temp'] = ddf['ticker'].str.replace('BANKNIFTY','')
    ddf['Temp'] = ddf['Temp'].str[:-2]
    ddf['Length_of_temp'] = np.where(ddf['Temp'].str.len()==12,12,ddf['Temp'].str.len())
    ddf['Strike'] = np.where((ddf['Temp'].str.len()==12)|(ddf['Temp'].str.len()==10),ddf['Temp'].str[-5:],
                             ddf['Temp'].str[-4:])
    ddf['Exp_year'] = np.where(ddf['Temp'].str.len()==12,ddf['Temp'].str[5:7],ddf['Temp'].str[:2])
    ddf['Exp_month'] = ddf['Temp'].str[2:5]
    ddf['Exp_year'] = ddf['Exp_year'].astype('str')
    ddf['MonthYear'] = ddf['Exp_month'] + ddf['Exp_year']
    merged_df = pd.merge(ddf,exp_date,on='MonthYear')
    merged_df = merged_df.drop(['MonthYear','Month','Year'],axis=1)
    merged_df['Length_of_temp'] = merged_df['Length_of_temp'].astype('int64')
    df_10 = merged_df[(merged_df['Length_of_temp']==10) | (merged_df['Length_of_temp']==9)]
    df_12 = merged_df[merged_df['Length_of_temp']==12]
    df_12['DateDate'] = df_12['Temp'].str[:2]
    df_12['DateDate'] = df_12['DateDate'].astype('int64')
    df_12['Exp_DT'] = pd.to_datetime(merged_df['Exp_DT'],dayfirst=True)
    df_12['Exp_Day'] = df_12['Exp_DT'].dt.day
    df_12 = df_12[df_12['Exp_Day']==df_12['DateDate']]
    df_12 = df_12.drop(['DateDate','Exp_Day'],axis=1)

    ddf = df_10.append(df_12,ignore_index=True)
    ddf['time'] = ddf['time'].str.replace(' 15:00:59','15:00:59')
    ddf['time'] = ddf['time'].str.replace(' 9:','09:',regex=True)
    ddf['time'] = pd.to_datetime(ddf['time'], format='%H:%M:%S').dt.time
    ddf = ddf[(ddf['time']>=start_time) & (ddf['time']<=end_time)]
    ddf['exp_month_number'] = ddf.apply(lambda row : add(row["Exp_month"]), axis = 1)
    ddf['New_date'] = ddf['date']
    ddf["New_date"] = pd.to_datetime(ddf["New_date"],dayfirst=True)
    ddf["current_month_number"] = ddf['New_date'].dt.month
    ddf["difference"] = ddf['exp_month_number'].astype(int) - ddf["current_month_number"].astype(int)
    df1 = pd.merge(ddf, 
                     exp_df, 
                     on ='New_date', 
                     how ='left')
    df1.drop(df1.filter(regex="Unname"),axis=1, inplace=True)
    df1["current_exp_month_number"] = df1['curr_exp_date'].dt.month
    df1["Diff_months"] = df1["current_exp_month_number"] - df1["current_month_number"]
    df1["Diff_months"] = df1["Diff_months"].astype(int) 
    bdf = df1[df1["Diff_months"] == 0]
    adf = df1[(df1["Diff_months"] == 1) | (df1["Diff_months"] == -11)]
    if bdf.shape[0] + adf.shape[0] == df1.shape[0]:
        print("Sanity Check Success")
    else:
        print("Error1")
    agb = adf.groupby(["difference"])
    unique_val_list_a = list(adf["difference"].unique())
    bgb = bdf.groupby(["difference"])
    unique_val_list_b = list(bdf["difference"].unique())
    
    ## REMOVING YESTERDAY'S CREATED FILE
    if os.path.exists(folpath+sym+'-I.csv'):
        os.remove(folpath+sym+'-I.csv')
    if os.path.exists(folpath+sym+'-II.csv'):
        os.remove(folpath+sym+'-II.csv')
    if os.path.exists(folpath+sym+'-III.csv'):
        os.remove(folpath+sym+'-III.csv')
    if os.path.exists(folpath+sym+'misc.csv'):
        os.remove(folpath+sym+'misc.csv')
        
    for i in unique_val_list_b:
        temp_df_new = bgb.get_group(i)
        temp_df_new = temp_df_new.drop(temp_df_new.columns[9:],axis=1)
        if i == 0:
            temp_df_new.to_csv(folpath + sym + '-I.csv', mode = 'a', header = not os.path.exists(folpath + sym + '-I.csv'), index=False)
        if i == 1 or i == -11:
            temp_df_new.to_csv(folpath + sym + '-II.csv', mode = 'a', header = not os.path.exists(folpath + sym + '-II.csv'), index=False)
        if i == 2 or i == -10:
            temp_df_new.to_csv(folpath + sym + '-III.csv', mode = 'a', header = not os.path.exists(folpath + sym + '-III.csv'), index=False)
        
    for i in unique_val_list_a:
        temp_df_new = agb.get_group(i)
        temp_df_new = temp_df_new.drop(temp_df_new.columns[9:],axis=1)
        if i == 1 or i == -11:
            temp_df_new.to_csv(folpath + sym + '-I.csv', mode = 'a', header = not os.path.exists(folpath + sym + '-I.csv'), index=False)
        if i == 2 or i == -10:
            temp_df_new.to_csv(folpath + sym + '-II.csv', mode = 'a', header = not os.path.exists(folpath + sym + '-II.csv'), index=False)
        if i == 3 or i == -9:
            temp_df_new.to_csv(folpath + sym + '-III.csv', mode = 'a', header = not os.path.exists(folpath + sym + '-III.csv'), index=False)
    
    ##########################CREATING LABEL IN STANDARD FORM#####################
    for i in range(3):
        if i == 0:
            file='I'
        elif i == 1:
            file='II'
        elif i == 2:
            file='III'
        
        if os.path.exists(r'C:\users\admin\desktop\Pyspark\BankNifty\Monthly\Banknifty-'+file+".csv"):
            os.remove(r'C:\users\admin\desktop\Pyspark\BankNifty\Monthly\Banknifty-'+file+".csv")
    
        if os.path.exists(r'C:\users\admin\desktop\Pyspark_Contracts\BankNifty\Monthly_Data\BANKNIFTY-'+file+'.csv'):
            ddf = pd.read_csv(r'C:\users\admin\desktop\Pyspark_Contracts\BankNifty\Monthly_Data\BANKNIFTY-'+file+'.csv')
            ddf['Option_Type'] = ddf['ticker'].str[-2:]
            ddf['Strike'] = np.where((ddf['ticker'].str.len()==20) | (ddf['ticker'].str.len()==22) , ddf['ticker'].str[-6:-2] , ddf['ticker'].str[-7:-2])
            ddf['Symbol'] = 'BANKNIFTY' + 'MONTHLY-' + file + ddf['Strike'].astype(int).astype(str) + ddf['Option_Type']
            ddf['ticker'] = ddf['Symbol']
            ddf = ddf.drop(ddf.columns[9:],axis=1)
            ddf = ddf.rename(columns = {"ticker":"Ticker"})
            ddf.to_csv(r"C:\Users\admin\Desktop\Pyspark\BankNifty\Monthly\\Banknifty-"+file+".csv",index=False)
    print("BANKNIFTY MONTHLY CONTRACTS CREATED")

def banknifty_weekly():
    ## CHECKING IF PATH DOES NOT EXIST, THEN CREATE PATH
    if not os.path.exists(r"C:\Users\admin\Desktop\Pyspark_Contracts\BankNifty\Weekly_data\\"):
        os.makedirs(r"C:\Users\admin\Desktop\Pyspark_Contracts\BankNifty\Weekly_data\\")
        
    if not os.path.exists(r"C:\users\admin\desktop\Pyspark\BankNifty\Weekly\\"):
        os.makedirs(r"C:\users\admin\desktop\Pyspark\BankNifty\Weekly\\")
        
    folpath = r"C:\users\admin\desktop\\Pyspark_Contracts\\BankNifty\\Weekly_Data\\"
    sym = 'BANKNIFTY'
    s = 'BANKNIFTY'
    expiry_time = datetime.strptime('15:29:59', '%H:%M:%S').time()
    
    ## CALLING FUNCTION NIFTY_DATA TO GENERATE ONLY NIFTY TICKERS
    df=banknifty_data()
    
    ## READING WEEKLY EXPIRY FILES
    exp_df = pd.read_csv(r"C:\Users\admin\Downloads\WeeklyExpiry.csv",parse_dates = ["date"],dayfirst =True,usecols= ['date', 'Week_number'])
    exp_date = pd.read_excel(r'C:\users\admin\desktop\Expiry_DT.xlsx',parse_dates = ['Exp_DT'],usecols = ['MonthYear','Exp_DT'])
    df['date'] = pd.to_datetime(df['date'])
    df['time'] = df['time'].str.replace(' 15:00:59','15:00:59')
    df['time'] = df['time'].str.replace(' 9:','09:',regex=True)
    df['time'] = pd.to_datetime(df['time']).dt.time
    df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
    df = df[df['time'] <= expiry_time]
    df['EXPIRY_DT'] = df['ticker'].str[9:16]
    df['EXPIRY_DT'] = pd.to_datetime(df['EXPIRY_DT'],dayfirst=True)
    df['OPTION_TYP'] = df['ticker'].str[-2:]
    df['STRIKE_PR'] = np.where(df['ticker'].str.len()==23,df['ticker'].str[-7:-2],df['ticker'].str[-7:-2])
    df['Month'] = df['ticker'].str[11:14]
    df['Year'] = np.where(df['ticker'].str.len()==23,df['ticker'].str[14:16],df['ticker'].str[9:11])
    df['MonthYear'] = df['Month'] + df['Year']
    df = df.rename(columns={'EXPIRY_DT' : 'expiry_date'})

    ## MERGING WITH EXPIRY SHEET TO GET EXPIRY DATE
    df1 = pd.merge(df,exp_df,on='date',how='left')
    df1 = df1.drop_duplicates()
    df1 = pd.merge(df1,exp_date,on='MonthYear',how='left')
    df1 = df1.drop(['Month','Year','MonthYear'],axis=1)

    ## GETTING THE EXPIRY DATES FOR MONTHLY CONTRACTS
    df1['expiry_date'] = np.where(df1['ticker'].str.len()>21,df1['expiry_date'],df1['Exp_DT'])
    df1 = df1.drop(['Exp_DT'],axis=1)
    exp_df = pd.read_csv(r"C:\Users\admin\Downloads\WeeklyExpiry.csv",parse_dates = ["Weekly_Expiry_Date"],dayfirst =True,usecols= ['Weekly_Expiry_Date', 'Expiry_Week_number'])
    exp_df = exp_df.dropna()
    exp_df = exp_df.rename(columns = {'Weekly_Expiry_Date': 'expiry_date'})
    df2 = pd.merge(df1, exp_df, on ='expiry_date', how ='left')
    df2 = df2.drop_duplicates()
    df2['week_diff'] = df2['Expiry_Week_number'] - df2['Week_number']

    final_df = df2[(df2["OPTION_TYP"] == "CE") | (df2["OPTION_TYP"] == "PE") ]
    final_df["week_diff"] = final_df['week_diff'].replace(np.nan,10000)

    agb = final_df.groupby(["week_diff"])
    unique_val_list_a = list(final_df["week_diff"].unique())
    unique_val_list_a = sorted([a for a in unique_val_list_a if a>=0])[0:12]
    print(unique_val_list_a)

    ## CREATING -I,-II AND SO ON BASED ON THE WEEK DIFFERENCES

    if os.path.exists(folpath+sym+'-I.csv'):
        os.remove(folpath+sym+'-I.csv')
    if os.path.exists(folpath+sym+'-II.csv'):
        os.remove(folpath+sym+'-II.csv')
    if os.path.exists(folpath+sym+'-III.csv'):
        os.remove(folpath+sym+'-III.csv')
    if os.path.exists(folpath+sym+'-IV.csv'):
        os.remove(folpath+sym+'-IV.csv')
    if os.path.exists(folpath+sym+'-V.csv'):
        os.remove(folpath+sym+'-V.csv')
    if os.path.exists(folpath+sym+'-VI.csv'):
        os.remove(folpath+sym+'-VI.csv')
    if os.path.exists(folpath+sym+'-VII.csv'):
        os.remove(folpath+sym+'-VII.csv')
    if os.path.exists(folpath+sym+'-VIII.csv'):
        os.remove(folpath+sym+'-VIII.csv')
    if os.path.exists(folpath+sym+'-IX.csv'):
        os.remove(folpath+sym+'-IX.csv')
    if os.path.exists(folpath+sym+'-X.csv'):
        os.remove(folpath+sym+'-X.csv')
    if os.path.exists(folpath+sym+'-XI.csv'):
        os.remove(folpath+sym+'-XI.csv')
    if os.path.exists(folpath+sym+'-XII.csv'):
        os.remove(folpath+sym+'-XII.csv')  
    if os.path.exists(folpath+sym+'-XIII.csv'):
        os.remove(folpath+sym+'-XIII.csv')  
    if os.path.exists(folpath+sym+'-XIV.csv'):
        os.remove(folpath+sym+'-XIV.csv')  
        
    for i in sorted(unique_val_list_a):
        temp_df = agb.get_group(i)
        temp_df = temp_df.drop(temp_df.columns[9:],axis=1)
        temp_df = temp_df.drop_duplicates()
        if i == 0:
            temp_df.to_csv(folpath + s + '-I.csv', mode = 'a', header = not os.path.exists(folpath + s + '-I.csv'), index=False)
        if i == 1:
            temp_df.to_csv(folpath + s + '-II.csv', mode = 'a', header = not os.path.exists(folpath + s + '-II.csv'), index=False)
        if i == 2:
            temp_df.to_csv(folpath + s + '-III.csv', mode = 'a', header = not os.path.exists(folpath + s + '-III.csv'), index=False)
        if i == 3:
            temp_df.to_csv(folpath + s + '-IV.csv', mode = 'a', header = not os.path.exists(folpath + s + '-IV.csv'), index=False)
        if i == 4:
            temp_df.to_csv(folpath + s + '-V.csv', mode = 'a', header = not os.path.exists(folpath + s + '-V.csv'), index=False)
        if i == 5:
            temp_df.to_csv(folpath + s + '-VI.csv', mode = 'a', header = not os.path.exists(folpath + s + '-VI.csv'), index=False)
        if i == 6:
            temp_df.to_csv(folpath + s + '-VII.csv', mode = 'a', header = not os.path.exists(folpath + s + '-VII.csv'), index=False)
        if i == 7:
            temp_df.to_csv(folpath + s + '-VIII.csv', mode = 'a', header = not os.path.exists(folpath + s + '-VIII.csv'), index=False)
        if i == 8:
            temp_df.to_csv(folpath + s + '-IX.csv', mode = 'a', header = not os.path.exists(folpath + s + '-IX.csv'), index=False)
        if i == 9:
            temp_df.to_csv(folpath + s + '-X.csv', mode = 'a', header = not os.path.exists(folpath + s + '-X.csv'), index=False)
        if i == 10:
            temp_df.to_csv(folpath + s + '-XI.csv', mode = 'a', header = not os.path.exists(folpath + s + '-XI.csv'), index=False)
        if i == 11:
            temp_df.to_csv(folpath + s + '-XII.csv', mode = 'a', header = not os.path.exists(folpath + s + '-XII.csv'), index=False)
        if i == 12:
            temp_df.to_csv(folpath + s + '-XIII.csv', mode = 'a', header = not os.path.exists(folpath + s + '-XIII.csv'), index=False)
        if i == 13:
            temp_df.to_csv(folpath + s + '-XIV.csv', mode = 'a', header = not os.path.exists(folpath + s + '-XIV.csv'), index=False)
    
    #################LABELLING FILES IN STANDARD FORM######################
    for i in range(15):
        if i==0:
            file='I'
        elif i==1:
            file='II'
        elif i==2:
            file='III'
        elif i==3:
            file='IV'
        elif i==4:
            file='V'
        elif i==5:
            file='VI'
        elif i==6:
            file='VII'
        elif i==7:
            file='VIII'
        elif i==8:
            file='IX'
        elif i==9:
            file='X'
        elif i==10:
            file='XI'
        elif i==11:
            file='XII'
        elif i==12:
            file='XIII'
        elif i==13:
            file='XIV'
        if os.path.exists(r'C:\users\admin\desktop\Pyspark\BankNifty\Weekly\BANKNIFTY-'+file+'.csv'):
            os.remove(r'C:\users\admin\desktop\Pyspark\BankNifty\Weekly\BANKNIFTY-'+file+'.csv')
        if os.path.exists(r'C:\users\admin\desktop\Pyspark_Contracts\BankNifty\Weekly_Data\BANKNIFTY-'+file+'.csv'):
            ddf = pd.read_csv(r'C:\users\admin\desktop\Pyspark_Contracts\BankNifty\Weekly_Data\BANKNIFTY-'+file+'.csv')
            ddf['Option_Type'] = ddf['ticker'].str[-2:]
            ddf['Strike'] = np.where((ddf['ticker'].str.len()==20) | (ddf['ticker'].str.len()==22) , ddf['ticker'].str[-6:-2] , ddf['ticker'].str[-7:-2])
            ddf['Symbol'] = 'BANKNIFTY' + 'WEEKLY-' + file + + ddf['Strike'].astype(int).astype(str) + ddf['Option_Type']
            ddf['ticker'] = ddf['Symbol']
            ddf = ddf.drop(ddf.columns[9:],axis=1)
            ddf = ddf.rename(columns = {'date':'Date','ticker':'Ticker'})
            ddf = ddf.drop_duplicates()
            ddf.to_csv(r"C:\Users\admin\Desktop\Pyspark\BankNifty\Weekly\BANKNIFTY-"+file+".csv",index=False)
    print("BANKNIFTY WEEKLY CONTRACTS CREATED")
    
def banknifty_quarterly():
    ## CHECKING IF PATH DOES NOT EXIST, THEN CREATE PATH
    if not os.path.exists(r"C:\Users\admin\Desktop\Pyspark_Contracts\BankNifty\Quarterly_data\\"):
        os.makedirs(r"C:\Users\admin\Desktop\Pyspark_Contracts\BankNifty\Quarterly_data\\")
        
    if not os.path.exists(r"C:\users\admin\desktop\Pyspark\BankNifty\Quarterly\\"):
        os.makedirs(r"C:\users\admin\desktop\Pyspark\BankNifty\Quarterly\\")
        
    folpath = r"C:\users\admin\desktop\\Pyspark_Contracts\\BankNifty\\Quarterly_Data\\"
    sym = 'BANKNIFTY'
    start_time = datetime.strptime('09:15:00', '%H:%M:%S').time()
    end_time = datetime.strptime('15:30:00', '%H:%M:%S').time()

    ## CALLING FUNCTION BANKNIFTY_DATA TO GENERATE ONLY BANKNIFTY TICKERS
    temp = banknifty_data()
    
    def add(stri):
        obj = datetime.strptime(stri, "%b")
        month_number = obj.month
        return month_number

    exp_date = pd.read_excel(r'C:\users\admin\desktop\Expiry_DT.xlsx')    ## reading the expiry sheet file
    exp_file_path = r"C:\Users\admin\Downloads\MonthlyExpiry.csv"
    exp_df = pd.read_csv(exp_file_path,parse_dates = ["curr_exp_date","curr_date"],dayfirst =True,usecols = ["curr_exp_date","curr_date"]).dropna()
    exp_df.rename({'curr_date': 'New_date'}, axis=1, inplace=True)
    temp['time'] = temp['time'].astype(str).str.replace(' 15:00:59','15:00:59')
    temp['time'] = temp['time'].str.replace(' 9:','09:',regex=True)
    temp['ticker'] = temp['ticker'].str.replace('30MAR23','29MAR23',regex=True)
    temp['time'] = pd.to_datetime(temp['time']).dt.time
    temp['date'] = pd.to_datetime(temp['date'])
    temp = temp[(temp['time']>=start_time) & (temp['time']<=end_time)]
    temp = temp.loc[:, ~temp.columns.str.contains('^Unnamed')]
    temp['Option_type'] = temp['ticker'].str[-2:]
    temp["Temp"] = temp["ticker"].str.replace('BANKNIFTY',"")
    temp["Temp"] = temp["Temp"].str[:-2]
    temp["Strike"] = np.where((temp['Temp'].str.len()==12) | (temp['Temp'].str.len()==10),
                                temp['Temp'].str[-5:],
                                temp['Temp'].str[-4:])
    temp['Current_Year'] = temp['date'].dt.year
    temp['Current_Year'] = temp['Current_Year'].astype(str).str[-2:]
    temp["Exp_year"] = np.where(temp['Temp'].str.len()==12,temp["Temp"].str[5:7],temp['Temp'].str[:2])
    temp["Exp_month"] = temp["Temp"].str[2:5]
    temp['Length_of_Temp'] = np.where(temp['Temp'].str.len()==12,12,temp['Temp'].str.len())
    temp['Exp_year'] = temp['Exp_year'].astype('str')
    temp['MonthYear'] = temp['Exp_month']+temp['Exp_year']
    temp = pd.merge(temp,exp_date,on='MonthYear')
    temp = temp.drop(['MonthYear','Month','Year'],axis=1)

    temp['Length_of_Temp'] = temp['Length_of_Temp'].astype('int64')
    temp_10 = temp[(temp['Length_of_Temp']==10) | (temp['Length_of_Temp']==9)]
    temp_12 = temp[temp['Length_of_Temp']==12]
    temp_12['datedate'] = temp_12['Temp'].str[:2]
    temp_12['datedate'] = temp_12['datedate'].astype('int64')
    temp_12['Exp_DT'] = pd.to_datetime(temp['Exp_DT'],dayfirst=True)
    temp_12['Exp_Day'] = temp_12['Exp_DT'].dt.day
    temp_12 = temp_12[temp_12['Exp_Day']==temp_12['datedate']]

    temp_12 = temp_12.drop(['datedate','Exp_Day'],axis=1)
    temp = temp_10.append(temp_12,ignore_index=True)

    temp['exp_month_number'] = temp.apply(lambda row : add(row["Exp_month"]), axis = 1)
    temp['New_date'] = temp['date']
    temp["New_date"] = pd.to_datetime(temp["New_date"])
    temp["current_month_number"] = temp['New_date'].dt.month
    temp["difference"] = temp['exp_month_number'].astype(int) - temp["current_month_number"].astype(int)
    temp['Year_difference'] = temp['Exp_year'].astype(int) - temp['Current_Year'].astype(int)
    temp = temp[(temp['exp_month_number']==3) | (temp['exp_month_number']==6) | (temp['exp_month_number']==9) | (temp['exp_month_number']==12)]

    temp1 = pd.merge(temp, 
                         exp_df, 
                         on ='New_date', 
                         how ='left')

    temp1.drop(temp1.filter(regex="Unname"),axis=1, inplace=True)
    temp1["current_exp_month_number"] = temp1['curr_exp_date'].dt.month
    temp1["Diff_months"] = temp1["current_exp_month_number"] - temp1["current_month_number"]
    temp1["Diff_months"] = temp1["Diff_months"].astype(int) 

    temp1 = temp1[temp1['Exp_DT']>=temp1['curr_exp_date']]                 ## to filter out dates which have wrong ticker

    ## creating groups for generating contracts
    atemp = temp1[(temp1['Diff_months']==0) & (temp1['Year_difference']==0)]
    agb = atemp.groupby(['difference'])
    unique_a = list(atemp['difference'].unique())

    if os.path.exists(folpath+sym+'-I.csv'):
        os.remove(folpath+sym+'-I.csv')
    if os.path.exists(folpath+sym+'-II.csv'):
        os.remove(folpath+sym+'-II.csv')
    if os.path.exists(folpath+sym+'-III.csv'):
        os.remove(folpath+sym+'-III.csv')
    if os.path.exists(folpath+sym+'-IV.csv'):
        os.remove(folpath+sym+'-IV.csv')
    if os.path.exists(folpath+sym+'-V.csv'):
        os.remove(folpath+sym+'-V.csv')
    if os.path.exists(folpath+sym+'-VI.csv'):
        os.remove(folpath+sym+'-VI.csv')
    if os.path.exists(folpath+sym+'-VII.csv'):
        os.remove(folpath+sym+'-VII.csv')
    if os.path.exists(folpath+sym+'-VIII.csv'):
        os.remove(folpath+sym+'-VIII.csv')    

    for i in unique_a:
        temp_df = agb.get_group(i)
        temp_df = temp_df.drop(temp_df.columns[9:],axis=1)
        if i==0 or i==1 or i==2:
            temp_df.to_csv(folpath + sym + '-I.csv', mode='a', header=not os.path.exists(folpath + sym + '-I.csv'), index=False)
        if i==3 or i==4 or i==5:
            temp_df.to_csv(folpath + sym + '-II.csv', mode='a', header=not os.path.exists(folpath + sym + '-II.csv'), index=False)
        if i==6 or i==7 or i==8:
            temp_df.to_csv(folpath + sym + '-III.csv', mode='a', header=not os.path.exists(folpath + sym + '-III.csv'), index=False)
        if i==9 or i==10 or i==11:
            temp_df.to_csv(folpath + sym + '-IV.csv', mode='a', header=not os.path.exists(folpath + sym + '-IV.csv'), index=False)

    btemp = temp1[(temp1['Diff_months']==0) & (temp1['Year_difference']==1)]
    bgb = btemp.groupby(['difference'])
    unique_b = list(btemp['difference'].unique())        

    for i in unique_b:
        temp_df = bgb.get_group(i)
        temp_df = temp_df.drop(temp_df.columns[9:],axis=1)
        if i==-7 or i==-8 or i==-9:
            temp_df.to_csv(folpath + sym + '-II.csv', mode='a', header=not os.path.exists(folpath + sym + '-II.csv'), index=False)
        if i==-4 or i==-5 or i==-6:
            temp_df.to_csv(folpath + sym + '-III.csv', mode='a', header=not os.path.exists(folpath + sym + '-III.csv'), index=False)
        if i==-1 or i==-2 or i==-3:
            temp_df.to_csv(folpath + sym + '-IV.csv', mode='a', header=not os.path.exists(folpath + sym + '-IV.csv'), index=False)
        if i==0 or i==1 or i==2:
            temp_df.to_csv(folpath + sym + '-V.csv', mode='a', header=not os.path.exists(folpath + sym + '-V.csv'), index=False)
        if i==3 or i==4 or i==5:
            temp_df.to_csv(folpath + sym + '-VI.csv', mode='a', header=not os.path.exists(folpath + sym + '-VI.csv'), index=False)
        if i==6 or i==7 or i==8:
            temp_df.to_csv(folpath + sym + '-VII.csv', mode='a', header=not os.path.exists(folpath + sym + '-VII.csv'), index=False)
        if i==9 or i==10 or i==11:
            temp_df.to_csv(folpath + sym + '-VIII.csv', mode='a', header=not os.path.exists(folpath + sym + '-VIII.csv'), index=False)

    ctemp = temp1[(temp1['Diff_months']==0) & (temp1['Year_difference']==2)]
    cgb = ctemp.groupby(['difference'])
    unique_c = list(ctemp['difference'].unique())        

    for i in unique_c:
        temp_df = cgb.get_group(i)
        temp_df = temp_df.drop(temp_df.columns[9:],axis=1)
        if i==-7 or i==-8 or i==-9:
            temp_df.to_csv(folpath + sym + '-VI.csv', mode='a', header=not os.path.exists(folpath + sym + '-VI.csv'), index=False)
        if i==-4 or i==-5 or i==-6:
            temp_df.to_csv(folpath + sym + '-VII.csv', mode='a', header=not os.path.exists(folpath + sym + '-VII.csv'), index=False)
        if i==-1 or i==-2 or i==-3:
            temp_df.to_csv(folpath + sym + '-VIII.csv', mode='a', header=not os.path.exists(folpath + sym + '-VIII.csv'), index=False)

    dtemp = temp1[((temp1['Diff_months']==1) | (temp1['Diff_months']==-11)) & (temp1['Year_difference']==0)]
    dgb = dtemp.groupby(['difference'])
    unique_d = list(dtemp['difference'].unique())

    for i in unique_d:
        temp_df = dgb.get_group(i)
        temp_df = temp_df.drop(temp_df.columns[9:],axis=1)
        if i==1 or i==2 or i==3:
            temp_df.to_csv(folpath + sym + '-I.csv', mode='a', header=not os.path.exists(folpath + sym + '-I.csv'), index=False)
        if i==4 or i==5 or i==6:
            temp_df.to_csv(folpath + sym + '-II.csv', mode='a', header=not os.path.exists(folpath + sym + '-II.csv'), index=False)
        if i==7 or i==8 or i==9:
            temp_df.to_csv(folpath + sym + '-III.csv', mode='a', header=not os.path.exists(folpath + sym + '-III.csv'), index=False)
        if i==10 or i==11:
            temp_df.to_csv(folpath + sym + '-IV.csv', mode='a', header=not os.path.exists(folpath + sym + '-IV.csv'), index=False)

    etemp = temp1[((temp1['Diff_months']==1) | (temp1['Diff_months']==-11)) & (temp1['Year_difference']==1)]
    egb = etemp.groupby(['difference'])
    unique_e = list(etemp['difference'].unique())

    for i in unique_e:
        temp_df = egb.get_group(i)
        temp_df = temp_df.drop(temp_df.columns[9:],axis=1)
        if i==-9:
            temp_df.to_csv(folpath + sym + '-I.csv', mode='a', header=not os.path.exists(folpath + sym + '-I.csv'), index=False)
        if i==-6 or i==-7:
            temp_df.to_csv(folpath + sym + '-II.csv', mode='a', header=not os.path.exists(folpath + sym + '-II.csv'), index=False)
        if i==-3 or i==-4:
            temp_df.to_csv(folpath + sym + '-III.csv', mode='a', header=not os.path.exists(folpath + sym + '-III.csv'), index=False)
        if i==0 or i==-1 or i==-2:
            temp_df.to_csv(folpath + sym + '-IV.csv', mode='a', header=not os.path.exists(folpath + sym + '-IV.csv'), index=False)
        if i==1 or i==2 or i==3:
            temp_df.to_csv(folpath + sym + '-V.csv', mode='a', header=not os.path.exists(folpath + sym + '-V.csv'), index=False)
        if i==4 or i==5 or i==6:
            temp_df.to_csv(folpath + sym + '-VI.csv', mode='a', header=not os.path.exists(folpath + sym + '-VI.csv'), index=False)
        if i==7 or i==8 or i==9:
            temp_df.to_csv(folpath + sym + '-VII.csv', mode='a', header=not os.path.exists(folpath + sym + '-VII.csv'), index=False)
        if i==10 or i==11:
            temp_df.to_csv(folpath + sym + '-VIII.csv', mode='a', header=not os.path.exists(folpath + sym + '-VIII.csv'), index=False)

    ftemp = temp1[((temp1['Diff_months']==1) | (temp1['Diff_months']==-11)) & (temp1['Year_difference']==2)]
    fgb = ftemp.groupby(['difference'])
    unique_f = list(ftemp['difference'].unique())

    for i in unique_f:
        temp_df = fgb.get_group(i)
        temp_df = temp_df.drop(temp_df.columns[9:],axis=1)
        if i==-9:
            temp_df.to_csv(folpath + sym + '-V.csv', mode='a', header=not os.path.exists(folpath + sym + '-V.csv'), index=False)
        if i==-6 or i==-7:
            temp_df.to_csv(folpath + sym + '-VI.csv', mode='a', header=not os.path.exists(folpath + sym + '-VI.csv'), index=False)
        if i==-3 or i==-4:
            temp_df.to_csv(folpath + sym + '-VII.csv', mode='a', header=not os.path.exists(folpath + sym + '-VII.csv'), index=False)
        if i==0 or i==-1 or i==-2:
            temp_df.to_csv(folpath + sym + '-VIII.csv', mode='a', header=not os.path.exists(folpath + sym + '-VIII.csv'), index=False)
    
    ## CREATING THE TICKER AND REMOVING ADDITIONAL COLUMNS
    for i in range(4):
        if i==0:
            file='I'
        elif i==1:
            file='II'
        elif i==2:
            file='III'
        elif i==3:
            file='IV'
        if os.path.exists(r"C:\Users\admin\Desktop\Pyspark\BankNifty\Quarterly\Banknifty-"+file+".csv"):
            os.remove(r"C:\Users\admin\Desktop\Pyspark\BankNifty\Quarterly\Banknifty-"+file+".csv")
        if os.path.exists(r"C:\Users\admin\Desktop\Pyspark_Contracts\BankNifty\Quarterly_data\BANKNIFTY-"+file+".csv"):
            ddf = pd.read_csv(r"C:\Users\admin\Desktop\Pyspark_Contracts\BankNifty\Quarterly_data\BANKNIFTY-"+file+".csv")
            ddf['Option_Type'] = ddf['ticker'].str[-2:]
            ddf['Strike'] = np.where((ddf['ticker'].str.len()==20) | (ddf['ticker'].str.len()==22) , ddf['ticker'].str[-6:-2] , ddf['ticker'].str[-7:-2])
            ddf['Symbol'] = 'BANKNIFTY' + 'QUARTERLY-' + file + + ddf['Strike'].astype(int).astype(str) + ddf['Option_Type']
            ddf['ticker'] = ddf['Symbol']
            ddf = ddf.drop(ddf.columns[9:],axis=1)
            ddf = ddf.rename(columns = {'date':'Date','ticker':'Ticker'})
            ddf = ddf.drop_duplicates()
            ddf.to_csv(r"C:\Users\admin\Desktop\Pyspark\BankNifty\Quarterly\Banknifty-"+file+".csv",index=False)
    print("BANKNIFTY QUARTERLY CONTRACTS CREATED")
    
def banknifty_halfyearly():
    ## CHECKING IF PATH DOES NOT EXIST, THEN CREATE PATH
    if not os.path.exists(r"C:\users\admin\desktop\Pyspark\BankNifty\Half_Yearly\\"):
        os.makedirs(r"C:\users\admin\desktop\Pyspark\BankNifty\Half_Yearly\\")
    
    if os.path.exists(r"C:\Users\admin\Desktop\Pyspark\BankNifty\\Half_Yearly\\Banknifty-I.csv"):
        os.remove(r"C:\Users\admin\Desktop\Pyspark\BankNifty\\Half_Yearly\\Banknifty-I.csv")

    if os.path.exists(r"C:\Users\admin\Desktop\Pyspark\BankNifty\\Half_Yearly\\Banknifty-II.csv"):
        os.remove(r"C:\Users\admin\Desktop\Pyspark\BankNifty\\Half_Yearly\\Banknifty-II.csv")
        
    final_df = pd.DataFrame()
    file = 'I'
    for i in range(2):
        df = pd.read_csv(r"C:\Users\admin\Desktop\Pyspark_Contracts\BankNifty\\Quarterly_Data\BANKNIFTY-"+str(file)+".csv")
        temp = df.copy()
        temp = temp.rename(columns = {'ticker':'Ticker','date':'Date','time':'Time','open':'Open','low':'Low','high':'High','close':'Close','volume':'Volume'})
        temp['Time'] = pd.to_datetime(temp['Time']).dt.time
        temp['Date'] = pd.to_datetime(temp['Date'])
        temp = temp.loc[:, ~temp.columns.str.contains('^Unnamed')]
        temp['Option_type'] = temp['Ticker'].str[-2:]
        temp["Temp"] = temp["Ticker"].str.replace('BANKNIFTY',"")
        temp["Temp"] = temp["Temp"].str[:-2]
        temp["Strike"] = np.where((temp['Temp'].str.len()==12) | (temp['Temp'].str.len()==10),
                                    temp['Temp'].str[-5:],
                                    temp['Temp'].str[-4:])
        temp['Current_Year'] = temp['Date'].dt.year
        temp['Current_Year'] = temp['Current_Year'].astype(str).str[-2:]
        temp["Exp_year"] = np.where(temp['Temp'].str.len()==12,temp["Temp"].str[5:7],temp['Temp'].str[:2])
        temp["Exp_month"] = temp["Temp"].str[2:5]
        temp['Length_of_Temp'] = np.where(temp['Temp'].str.len()==12,12,temp['Temp'].str.len())
        temp = temp[(temp['Exp_month']=='JUN') | (temp['Exp_month']=='DEC')]
        temp = temp.reset_index(drop=True)
        final_df = final_df.append(temp)
        final_df = final_df.reset_index(drop=True)
        final_df = final_df.drop(final_df.columns[9:],axis=1)
        file+=file

    if final_df.empty==False:    
        test = final_df.copy()
        test['Option_Type'] = test['Ticker'].str[-2:]
        test['Last'] = test['Ticker'].str[-7:]
        test['Strike'] = test['Last'].astype('str').str.extractall('(\d+)').unstack().fillna('').sum(axis=1).astype(int)
        test['Symbol'] = test['Ticker'].str[:9] + '-I' + test['Strike'].astype(str) + test['Option_Type'].astype(str)
        test['Ticker'] = test['Symbol']
        test = test.drop(test.columns[9:13],axis=1)
        print("BANKNIFTY HALF-YEARLY-I CREATED")
        test.to_csv(r"C:\Users\admin\Desktop\Pyspark\BankNifty\\Half_Yearly\\Banknifty-I.csv",index=False)


    final_df = pd.DataFrame()
    file = 'III'
    for i in range(2):
        if os.path.exists(r"C:\Users\admin\Desktop\Pyspark_Contracts\BankNifty\\Quarterly_Data\BANKNIFTY-"+str(file)+".csv"):
            df = pd.read_csv(r"C:\Users\admin\Desktop\Pyspark_Contracts\BankNifty\\Quarterly_Data\BANKNIFTY-"+str(file)+".csv")
            temp = df.copy()
            temp = temp.rename(columns = {'ticker':'Ticker','date':'Date','time':'Time','open':'Open','low':'Low','high':'High','close':'Close','volume':'Volume'})
            temp['Time'] = pd.to_datetime(temp['Time']).dt.time
            temp['Date'] = pd.to_datetime(temp['Date'])
            temp = temp.loc[:, ~temp.columns.str.contains('^Unnamed')]
            temp['Option_type'] = temp['Ticker'].str[-2:]
            temp["Temp"] = temp["Ticker"].str.replace('BANKNIFTY',"")
            temp["Temp"] = temp["Temp"].str[:-2]
            temp["Strike"] = np.where((temp['Temp'].str.len()==12) | (temp['Temp'].str.len()==10),
                                        temp['Temp'].str[-5:],
                                        temp['Temp'].str[-4:])
            temp['Current_Year'] = temp['Date'].dt.year
            temp['Current_Year'] = temp['Current_Year'].astype(str).str[-2:]
            temp["Exp_year"] = np.where(temp['Temp'].str.len()==12,temp["Temp"].str[5:7],temp['Temp'].str[:2])
            temp["Exp_month"] = temp["Temp"].str[2:5]
            temp['Length_of_Temp'] = np.where(temp['Temp'].str.len()==12,12,temp['Temp'].str.len())
            temp = temp[(temp['Exp_month']=='JUN') | (temp['Exp_month']=='DEC')]
            temp = temp.reset_index(drop=True)
            final_df = final_df.append(temp)
            final_df = final_df.reset_index(drop=True)
            final_df = final_df.drop(final_df.columns[9:],axis=1)
            file='IV'
        else:
            print("BANKNIFTY-QUARTERLY-"+str(file)+' not found')
            file='IV'

    if final_df.empty==False:
        test = final_df.copy()
        test['Option_Type'] = test['Ticker'].str[-2:]
        test['Last'] = test['Ticker'].str[-7:]
        test['Strike'] = test['Last'].astype('str').str.extractall('(\d+)').unstack().fillna('').sum(axis=1).astype(int)
        test['Symbol'] = test['Ticker'].str[:9] + '-II' + test['Strike'].astype(str) + test['Option_Type'].astype(str)
        test['Ticker'] = test['Symbol']
        test = test.drop(test.columns[9:13],axis=1)
        print("BANKNIFTY HALF-YEARLY-II CREATED")
        test.to_csv(r"C:\Users\admin\Desktop\Pyspark\BankNifty\\Half_Yearly\\Banknifty-II.csv",index=False)
    print("BANKNIFTY HALFYEARLY CONTRACTS CREATED")

def banknifty_yearly():
    ## CHECKING IF PATH DOES NOT EXIST, THEN CREATE PATH
    if not os.path.exists(r"C:\users\admin\desktop\Pyspark\BankNifty\Yearly\\"):
        os.makedirs(r"C:\users\admin\desktop\Pyspark\BankNifty\Yearly\\")
        
    if os.path.exists(r"C:\Users\admin\Desktop\Pyspark\BankNifty\\Yearly\\Banknifty-I.csv"):
        os.remove(r"C:\Users\admin\Desktop\Pyspark\BankNifty\\Yearly\\Banknifty-I.csv")

    final_df = pd.DataFrame()
    file = 'I'
    add_file = 'I'
    for i in range(4):
        print(i,file)
        if os.path.exists(r"C:\Users\admin\Desktop\Pyspark_Contracts\BankNifty\\Quarterly_Data\BANKNIFTY-"+str(file)+".csv"):  
            df = pd.read_csv(r"C:\Users\admin\Desktop\Pyspark_Contracts\BankNifty\\Quarterly_Data\BANKNIFTY-"+str(file)+".csv")
            temp = df.copy()
            temp = temp.rename(columns = {'ticker':'Ticker','date':'Date','time':'Time','open':'Open','low':'Low','high':'High','close':'Close','volume':'Volume'})
            temp['Time'] = pd.to_datetime(temp['Time']).dt.time
            temp['Date'] = pd.to_datetime(temp['Date'])
            temp = temp.loc[:, ~temp.columns.str.contains('^Unnamed')]
            temp['Option_type'] = temp['Ticker'].str[-2:]
            temp["Temp"] = temp["Ticker"].str.replace('BANKNIFTY',"")
            temp["Temp"] = temp["Temp"].str[:-2]
            temp["Strike"] = np.where((temp['Temp'].str.len()==12) | (temp['Temp'].str.len()==10),
                                        temp['Temp'].str[-5:],
                                        temp['Temp'].str[-4:])
            temp['Current_Year'] = temp['Date'].dt.year
            temp['Current_Year'] = temp['Current_Year'].astype(str).str[-2:]
            temp["Exp_year"] = np.where(temp['Temp'].str.len()==12,temp["Temp"].str[5:7],temp['Temp'].str[:2])
            temp["Exp_month"] = temp["Temp"].str[2:5]
            temp['Length_of_Temp'] = np.where(temp['Temp'].str.len()==12,12,temp['Temp'].str.len())
            temp = temp[(temp['Exp_month']=='DEC')]
            temp = temp.reset_index(drop=True)
            final_df = final_df.append(temp)
            final_df = final_df.reset_index(drop=True)
            final_df = final_df.drop(final_df.columns[9:],axis=1)
            if i==2:
                file='IV'
            else:
                file+=add_file

        else:
            print("BANKNIFTY QUARTERLY-"+str(file)+" does not exist")

    if final_df.empty==False:
        test = final_df.copy()
        test['Option_Type'] = test['Ticker'].str[-2:]
        test['Last'] = test['Ticker'].str[-7:]
        test['Strike'] = test['Last'].astype('str').str.extractall('(\d+)').unstack().fillna('').sum(axis=1).astype(int)
        test['Symbol'] = test['Ticker'].str[:9] + '-I' + test['Strike'].astype(str) + test['Option_Type'].astype(str)
        test['Ticker'] = test['Symbol']
        test = test.drop(test.columns[9:13],axis=1)
        print("BANKNIFTY YEARLY-I CREATED")
        test.to_csv(r"C:\Users\admin\Desktop\Pyspark\BankNifty\\Yearly\\Banknifty-I.csv",index=False)

    else:
        print("Dataframe is empty!!!")
    print("BANKNIFTY YEARLY CONTRACTS CREATED")

def nifty_data():
    spark = SparkSession.builder.config("spark.jars", "C:\\Users\\admin\\Downloads\\postgresql-42.5.0.jar") \
    .master("local").appName("PySpark_Postgres_test").getOrCreate()
    
    df = spark.read.format("jdbc").option("url", "jdbc:postgresql://swandatabase.cfehmk2wtejq.ap-south-1.rds.amazonaws.com/RawDataBase").option("user","postgres").option("password","swancap123")\
        .option("driver", "org.postgresql.Driver").option("dbtable", tablename)\
        .option("user", "postgres").option("password", "swancap123").load()

    ## GETTING ONLY TIME IN TIME COLUMN
    q = df.withColumn('time',date_format('time', 'HH:mm:ss'))
    ndata = q.filter(q.ticker.contains('NIFTY') & ((q.ticker.endswith('E.NFO'))| (q.ticker.endswith('E'))))
    ndata = ndata.withColumn('ticker_check',substring('ticker',1,5))
    ndata = ndata.filter(ndata.ticker_check.contains('NIFTY'))
    ## REPLACING .NFO IN ticker
    ndata = ndata.withColumn('ticker',regexp_replace('ticker','.NFO',''))
    ndata = ndata.drop(col('ticker_check'))

    ## CONVERTING PYSPARK DATAFRAME TO PANDAS DATAFRAME
    ndata = ndata.toPandas()
    return ndata
    
def nifty_monthly():
    folpath = r"C:\Users\admin\Desktop\Pyspark_Contracts\Nifty\Monthly_Data\\"
    sym = 'NIFTY'
    start_time = datetime.strptime('09:15:00', '%H:%M:%S').time()
    end_time = datetime.strptime('15:30:00', '%H:%M:%S').time()
    expiry_time = datetime.strptime('15:29:59', '%H:%M:%S').time()
    s = 'NIFTY'

    def add(stri):
        obj = datetime.strptime(stri, "%b")
        month_number = obj.month
        return month_number

    def get_symbol(tic):
        li = list(filter(None, re.split(r'(\d+)', tic)))
        return li[0]

    exp_file_path = r"C:\Users\admin\Downloads\MonthlyExpiry.csv"
    exp_df = pd.read_csv(exp_file_path,parse_dates = ["curr_exp_date","curr_date"],dayfirst =True).dropna()
    exp_df.rename({'curr_date': 'New_date'}, axis=1, inplace=True)
    exp_date = pd.read_excel(r'C:\users\admin\desktop\Expiry_DT.xlsx')    ## reading the expiry sheet file
    
    ## CALLING FUNCTION NIFTY_DATA TO GENERATE ONLY NIFTY TICKERS
    ndata = nifty_data()
    
    temp = ndata.copy()
    temp = temp.loc[:, ~temp.columns.str.contains('^Unnamed')]
    temp['Symbol'] = temp['ticker'].str[:7]
    temp = temp[temp['Symbol']!='NIFTYIT']
    temp['time'] = pd.to_datetime(temp['time']).dt.time
    temp['ticker'] = temp['ticker'].str.replace('30MAR23','29MAR23',regex=True)
    temp['Option_Type'] = temp['ticker'].str[-2:]
    temp['Temp'] = temp["ticker"].str.replace(s,"")
    temp['Temp'] = temp['Temp'].str[:-2]
    temp['Length_of_temp'] = temp['Temp'].str.len()
    temp['Strike'] = np.where((temp['Temp'].str.len()==9) | (temp['Temp'].str.len()==11) , 
                              temp['Temp'].str[-4:] , 
                              temp['Temp'].str[-5:])
    temp['Exp_Year'] = np.where((temp['Temp'].str.len()==9) | (temp['Temp'].str.len()==10) ,
                               temp['Temp'].str[:2] ,
                               temp['Temp'].str[5:7])
    temp['Exp_month'] = temp['Temp'].str[2:5]
    temp['Exp_Year'] = temp['Exp_Year'].astype('str')
    temp['MonthYear'] = temp['Exp_month']+temp['Exp_Year']
    temp = pd.merge(temp,exp_date,on='MonthYear')
    temp = temp.drop(['MonthYear','Month','Year','Next_Exp_DT'],axis=1)
    temp['Length_of_temp'] = temp['Length_of_temp'].astype('int64')
    temp_10 = temp[(temp['Length_of_temp']==10) | (temp['Length_of_temp']==9)]

    temp_12 = temp[(temp['Length_of_temp']==12) | (temp['Length_of_temp']==11)]
    temp_12['DateDate'] = temp_12['Temp'].str[:2]
    temp_12['DateDate'] = temp_12['DateDate'].astype('int64')
    temp_12['Exp_DT'] = pd.to_datetime(temp_12['Exp_DT'],dayfirst=True)
    temp_12['Exp_Day'] = temp_12['Exp_DT'].dt.day
    temp_12 = temp_12[temp_12['Exp_Day']==temp_12['DateDate']]
    temp_12 = temp_12.drop(['DateDate','Exp_Day'],axis=1)

    temp_df = temp_10.append(temp_12,ignore_index=True)

    temp_df['time'] = temp_df['time'].astype(str).str.replace(' 15:00:59','15:00:59')
    temp_df['time'] = temp_df['time'].astype(str).str.replace(' 9:','09:',regex=True)
    temp_df['time'] = pd.to_datetime(temp_df['time'], format='%H:%M:%S').dt.time
    temp_df = temp_df[(temp_df['time']>=start_time) & (temp_df['time']<=end_time)]

    temp_df['exp_month_number'] = temp_df.apply(lambda row : add(row["Exp_month"]), axis = 1)
    temp_df['New_date'] = temp_df['date']
    temp_df["New_date"] = pd.to_datetime(temp_df["New_date"])
    temp_df["current_month_number"] = temp_df['New_date'].dt.month
    temp_df["difference"] = temp_df['exp_month_number'].astype(int) - temp_df["current_month_number"].astype(int)
    temp_df1 = pd.merge(temp_df, 
                 exp_df, 
                 on ='New_date', 
                 how ='left')
    temp_df1.drop(temp_df1.filter(regex="Unname"),axis=1, inplace=True)
    temp_df1["current_exp_month_number"] = temp_df1['curr_exp_date'].dt.month
    temp_df1["Diff_months"] = temp_df1["current_exp_month_number"] - temp_df1["current_month_number"]
    temp_df1["Diff_months"] = temp_df1["Diff_months"].astype(int) 
    temp_df1['Current_Year'] = temp_df1['New_date'].dt.year.astype(str).str[-2:]
    temp_df1['Flag'] = np.where((temp_df1['Current_Year']==temp_df1['Exp_Year']) | (temp_df1['current_month_number']==12) & (temp_df1['exp_month_number']<=3),1,0)
    temp_df1 = temp_df1[temp_df1['Flag']==1]
    bdf = temp_df1[temp_df1["Diff_months"] == 0]
    adf = temp_df1[(temp_df1["Diff_months"] == 1) | (temp_df1["Diff_months"] == -11)]
    if bdf.shape[0] + adf.shape[0] == temp_df1.shape[0]:
        print("Sanity Check Success")
    else:
        print("Error")

    agb = adf.groupby(["difference"])
    unique_val_list_a = list(adf["difference"].unique())
    bgb = bdf.groupby(["difference"])
    unique_val_list_b = list(bdf["difference"].unique())

    if os.path.exists(folpath+sym+'-I.csv'):
        os.remove(folpath+sym+'-I.csv')
    if os.path.exists(folpath+sym+'-II.csv'):
        os.remove(folpath+sym+'-II.csv')
    if os.path.exists(folpath+sym+'-III.csv'):
        os.remove(folpath+sym+'-III.csv')
    if os.path.exists(folpath+sym+'-misc.csv'):
        os.remove(folpath+sym+'-misc.csv')

    for i in unique_val_list_b:
        temp_df_new = bgb.get_group(i)
        temp_df_new = temp_df_new.drop(temp_df_new.columns[9:],axis=1)
        if i == 0:
            temp_df_new.to_csv(folpath + sym + '-I.csv', mode = 'a', header = not os.path.exists(folpath + sym + '-I.csv'), index=False)

        if i == 1 or i == -11:
            temp_df_new.to_csv(folpath + sym + '-II.csv', mode = 'a', header = not os.path.exists(folpath + sym + '-II.csv'), index=False)

        if i == 2 or i == -10:
            temp_df_new.to_csv(folpath + sym + '-III.csv', mode = 'a', header = not os.path.exists(folpath + sym + '-III.csv'), index=False)

    for i in unique_val_list_a:
        temp_df_new = agb.get_group(i)
        temp_df_new = temp_df_new.drop(temp_df_new.columns[9:],axis=1)
        if i == 1 or i == -11:
            temp_df_new.to_csv(folpath + sym + '-I.csv', mode = 'a', header = not os.path.exists(folpath + sym + '-I.csv'), index=False)

        if i == 2 or i == -10:
            temp_df_new.to_csv(folpath + sym + '-II.csv', mode = 'a', header = not os.path.exists(folpath + sym + '-II.csv'), index=False)

        if i == 3 or i == -9:
            temp_df_new.to_csv(folpath + sym + '-III.csv', mode = 'a', header = not os.path.exists(folpath + sym + '-III.csv'), index=False)

    ##########################CREATING LABEL IN STANDARD FORM#####################
    for i in range(3):
        if i == 0:
            file='I'
        elif i == 1:
            file='II'
        elif i == 2:
            file='III'
        if os.path.exists(r'C:\users\admin\desktop\Pyspark\Nifty\Monthly\NIFTY-'+file+".csv"):
            os.remove(r'C:\users\admin\desktop\Pyspark\Nifty\Monthly\NIFTY-'+file+".csv")
        if os.path.exists(r'C:\users\admin\desktop\Pyspark_Contracts\Nifty\Monthly_Data\NIFTY-'+file+'.csv'):
            ddf = pd.read_csv(r'C:\users\admin\desktop\Pyspark_Contracts\Nifty\Monthly_Data\NIFTY-'+file+'.csv')
            ddf['Option_Type'] = ddf['ticker'].str[-2:]
            ddf['Strike'] = np.where((ddf['ticker'].str.len()==16) | (ddf['ticker'].str.len()==18) , ddf['ticker'].str[-6:-2] , ddf['ticker'].str[-7:-2])
            ddf['Symbol'] = 'NIFTY' + 'MONTHLY-' + file + ddf['Strike'].astype(int).astype(str) + ddf['Option_Type']
            ddf['ticker'] = ddf['Symbol']
            ddf = ddf.drop(ddf.columns[9:],axis=1)
            ddf = ddf.rename(columns = {"ticker":"Ticker"})
            ddf.to_csv(r"C:\Users\admin\Desktop\Pyspark\Nifty\Monthly\\NIFTY-"+file+".csv",index=False)
    print("NIFTY MONTHLY CONTRACTS CREATED")

def nifty_weekly():
    folpath = r"C:\Users\admin\Desktop\Pyspark_Contracts\Nifty\Weekly_Data\\"
    sym = 'NIFTY'
    start_time = datetime.strptime('09:15:00', '%H:%M:%S').time()
    end_time = datetime.strptime('15:30:00', '%H:%M:%S').time()
    expiry_time = datetime.strptime('15:29:59', '%H:%M:%S').time()
    s = 'NIFTY'
    def add(stri):
        obj = datetime.strptime(stri, "%b")
        month_number = obj.month
        return month_number
    def get_symbol(tic):
        li = list(filter(None, re.split(r'(\d+)', tic)))
        return li[0]
    exp_file_path = r"C:\Users\admin\Downloads\WeeklyExpiry.csv"
    exp_date = pd.read_excel(r'C:\users\admin\desktop\Expiry_DT.xlsx')    ## reading the expiry sheet file
    exp_df = pd.read_csv(exp_file_path,parse_dates = ["date"],dayfirst =True,usecols= ['date', 'Week_number'])
    exp_df.rename({'curr_date': 'New_date'}, axis=1, inplace=True)
    
    ## CALLING FUNCTION NIFTY_DATA TO GENERATE ONLY NIFTY TICKERS
    ndata=nifty_data()
    
    temp = ndata.copy()
    temp = temp.loc[:, ~temp.columns.str.contains('^Unnamed')]
    temp['Symbol'] = temp['ticker'].str[:7]
    temp = temp[temp['Symbol']!='NIFTYIT']
    temp = temp.reset_index(drop=True)
    temp['time'] = temp['time'].str.replace(' 15:00:59','15:00:59')
    temp['time'] = temp['time'].str.replace(' 9:','09:',regex=True)
    temp['time'] = pd.to_datetime(temp['time']).dt.time
    temp['date'] = pd.to_datetime(temp['date'])
    temp['ticker'] = temp['ticker'].str.replace('30MAR23','29MAR23',regex=True)
    temp['Option_Type'] = temp['ticker'].str[-2:]
    temp['Temp'] = temp["ticker"].str.replace(s,"")
    temp['Temp'] = temp['Temp'].str[:-2]
    temp['EXPIRY_DT'] = temp['Temp'].str[:7]
    temp['EXPIRY_DT'] = pd.to_datetime(temp['EXPIRY_DT'],dayfirst=True)
    temp['Length_of_temp'] = temp['Temp'].str.len()
    temp['Strike'] = np.where((temp['Temp'].str.len()==9) | (temp['Temp'].str.len()==11) , 
                              temp['Temp'].str[-4:] , 
                              temp['Temp'].str[-5:])
    temp['Exp_year'] = np.where((temp['Temp'].str.len()==9) | (temp['Temp'].str.len()==10) ,
                               temp['Temp'].str[:2] ,
                               temp['Temp'].str[5:7])
    temp['Exp_month'] = temp['Temp'].str[2:5]
    temp['MonthYear'] = temp['Exp_month'] + temp['Exp_year']
    temp = temp.rename(columns={'EXPIRY_DT' : 'expiry_date'})
    temp1 = pd.merge(temp,exp_df,on='date',how='left')
    temp1 = temp1.drop_duplicates()
    temp1 = pd.merge(temp1,exp_date,on='MonthYear',how='left')
    temp1 = temp1.drop(['Month','Year','MonthYear','Next_Exp_DT'],axis=1)

    ## GETTING EXPIRY DATES FOR MONTHLY CONTRACTS
    temp1['expiry_date'] = np.where(temp1['Length_of_temp']>=11,temp1['expiry_date'],temp1['Exp_DT'])
    temp1 = temp1.drop(['Exp_DT'],axis=1)
    exp_df = pd.read_csv(exp_file_path,parse_dates = ["Weekly_Expiry_Date"],dayfirst =True,usecols= ['Weekly_Expiry_Date', 'Expiry_Week_number'])
    exp_df = exp_df.dropna()
    exp_df = exp_df.rename(columns = {'Weekly_Expiry_Date':'expiry_date'})
    temp2 = pd.merge(temp1, exp_df, on = 'expiry_date', how = 'left')
    temp2 = temp2.drop_duplicates()
    temp2['week_diff'] = temp2['Expiry_Week_number'] - temp2['Week_number']
    final_df = temp2
    final_df["week_diff"] = final_df['week_diff'].replace(np.nan,10000)

    agb = final_df.groupby(["week_diff"])
    unique_val_list_a = list(final_df["week_diff"].unique())
    unique_val_list_a = sorted([a for a in unique_val_list_a if a>=0])[0:12]
    print(unique_val_list_a)

    ############CREATING -I,-II BASED ON WEEK DIFFERENCES###################
    if os.path.exists(folpath+sym+'-I.csv'):
        os.remove(folpath+sym+'-I.csv')
    if os.path.exists(folpath+sym+'-II.csv'):
        os.remove(folpath+sym+'-II.csv')
    if os.path.exists(folpath+sym+'-III.csv'):
        os.remove(folpath+sym+'-III.csv')
    if os.path.exists(folpath+sym+'-IV.csv'):
        os.remove(folpath+sym+'-IV.csv')
    if os.path.exists(folpath+sym+'-V.csv'):
        os.remove(folpath+sym+'-V.csv')
    if os.path.exists(folpath+sym+'-VI.csv'):
        os.remove(folpath+sym+'-VI.csv')
    if os.path.exists(folpath+sym+'-VII.csv'):
        os.remove(folpath+sym+'-VII.csv')
    if os.path.exists(folpath+sym+'-VIII.csv'):
        os.remove(folpath+sym+'-VIII.csv')
    if os.path.exists(folpath+sym+'-IX.csv'):
        os.remove(folpath+sym+'-IX.csv')
    if os.path.exists(folpath+sym+'-X.csv'):
        os.remove(folpath+sym+'-X.csv')
    if os.path.exists(folpath+sym+'-XI.csv'):
        os.remove(folpath+sym+'-XI.csv')
    if os.path.exists(folpath+sym+'-XII.csv'):
        os.remove(folpath+sym+'-XII.csv')  
    if os.path.exists(folpath+sym+'-XIII.csv'):
        os.remove(folpath+sym+'-XIII.csv')  
    if os.path.exists(folpath+sym+'-XIV.csv'):
        os.remove(folpath+sym+'-XIV.csv')  
        
    for i in sorted(unique_val_list_a):
        temp_df = agb.get_group(i)
        temp_df = temp_df.drop(temp_df.columns[9:],axis=1)
        temp_df = temp_df.drop_duplicates()
        if i == 0:
            temp_df.to_csv(folpath + s + '-I.csv', mode = 'a', header = not os.path.exists(folpath + s + '-I.csv'), index=False)

        if i == 1:
            temp_df.to_csv(folpath + s + '-II.csv', mode = 'a', header = not os.path.exists(folpath + s + '-II.csv'), index=False)

        if i == 2:
            temp_df.to_csv(folpath + s + '-III.csv', mode = 'a', header = not os.path.exists(folpath + s + '-III.csv'), index=False)

        if i == 3:
            temp_df.to_csv(folpath + s + '-IV.csv', mode = 'a', header = not os.path.exists(folpath + s + '-IV.csv'), index=False)

        if i == 4:
            temp_df.to_csv(folpath + s + '-V.csv', mode = 'a', header = not os.path.exists(folpath + s + '-V.csv'), index=False)

        if i == 5:
            temp_df.to_csv(folpath + s + '-VI.csv', mode = 'a', header = not os.path.exists(folpath + s + '-VI.csv'), index=False)

        if i == 6:
            temp_df.to_csv(folpath + s + '-VII.csv', mode = 'a', header = not os.path.exists(folpath + s + '-VII.csv'), index=False)

        if i == 7:
            temp_df.to_csv(folpath + s + '-VIII.csv', mode = 'a', header = not os.path.exists(folpath + s + '-VIII.csv'), index=False)

        if i == 8:
            temp_df.to_csv(folpath + s + '-IX.csv', mode = 'a', header = not os.path.exists(folpath + s + '-IX.csv'), index=False)

        if i == 9:
            temp_df.to_csv(folpath + s + '-X.csv', mode = 'a', header = not os.path.exists(folpath + s + '-X.csv'), index=False)

        if i == 10:
            temp_df.to_csv(folpath + s + '-XI.csv', mode = 'a', header = not os.path.exists(folpath + s + '-XI.csv'), index=False)

        if i == 11:
            temp_df.to_csv(folpath + s + '-XII.csv', mode = 'a', header = not os.path.exists(folpath + s + '-XII.csv'), index=False)

        if i == 12:
            temp_df.to_csv(folpath + s + '-XIII.csv', mode = 'a', header = not os.path.exists(folpath + s + '-XIII.csv'), index=False)

        if i == 13:
            temp_df.to_csv(folpath + s + '-XIV.csv', mode = 'a', header = not os.path.exists(folpath + s + '-XIV.csv'), index=False)

    #################LABELLING FILES IN STANDARD FORM######################
    for i in range(15):
        if i==0:
            file='I'
        elif i==1:
            file='II'
        elif i==2:
            file='III'
        elif i==3:
            file='IV'
        elif i==4:
            file='V'
        elif i==5:
            file='VI'
        elif i==6:
            file='VII'
        elif i==7:
            file='VIII'
        elif i==8:
            file='IX'
        elif i==9:
            file='X'
        elif i==10:
            file='XI'
        elif i==11:
            file='XII'
        elif i==12:
            file='XIII'
        elif i==13:
            file='XIV'
        if os.path.exists(r'C:\users\admin\desktop\Pyspark\Nifty\Weekly\NIFTY-'+file+'.csv'):
            os.remove(r'C:\users\admin\desktop\Pyspark\Nifty\Weekly\NIFTY-'+file+'.csv')
        if os.path.exists(r'C:\users\admin\desktop\Pyspark_Contracts\Nifty\Weekly_Data\NIFTY-'+file+'.csv'):
            ddf = pd.read_csv(r'C:\users\admin\desktop\Pyspark_Contracts\Nifty\Weekly_Data\NIFTY-'+file+'.csv')
            ddf['Option_Type'] = ddf['ticker'].str[-2:]
            ddf['Strike'] = np.where((ddf['ticker'].str.len()==16) | (ddf['ticker'].str.len()==18) , ddf['ticker'].str[-6:-2] , ddf['ticker'].str[-7:-2])
            ddf['Symbol'] = 'NIFTY' + 'WEEKLY-' + file + + ddf['Strike'].astype(int).astype(str) + ddf['Option_Type']
            ddf['ticker'] = ddf['Symbol']
            ddf = ddf.drop(ddf.columns[9:],axis=1)
            ddf = ddf.rename(columns = {'date':'Date','ticker':'Ticker'})
            ddf = ddf.drop_duplicates()
            ddf.to_csv(r"C:\Users\admin\Desktop\Pyspark\Nifty\Weekly\NIFTY-"+file+".csv",index=False)
    print("NIFTY WEEKLY CONTRACTS CREATED")
    
def nifty_quarterly():
    folpath = r"C:\Users\admin\Desktop\Pyspark_Contracts\Nifty\Quarterly_Data\\"
    sym = 'NIFTY'
    start_time = datetime.strptime('09:15:00', '%H:%M:%S').time()
    end_time = datetime.strptime('15:30:00', '%H:%M:%S').time()
    expiry_time = datetime.strptime('15:29:59', '%H:%M:%S').time()
    s = 'NIFTY'

    def add(stri):
        obj = datetime.strptime(stri, "%b")
        month_number = obj.month
        return month_number

    def get_symbol(tic):
        li = list(filter(None, re.split(r'(\d+)', tic)))
        return li[0]

    exp_date = pd.read_excel(r'C:\users\admin\desktop\Expiry_DT.xlsx')    ## reading the expiry sheet file
    exp_file_path = r"C:\Users\admin\Downloads\MonthlyExpiry.csv"
    exp_df = pd.read_csv(exp_file_path,parse_dates = ["curr_exp_date","curr_date"],dayfirst =True,usecols = ["curr_exp_date","curr_date"]).dropna()
    exp_df.rename({'curr_date': 'New_date'}, axis=1, inplace=True)
    
    ndata = nifty_data()
    temp = ndata.copy()
    temp = temp.loc[:, ~temp.columns.str.contains('^Unnamed')]
    temp = temp.reset_index(drop=True)
    temp['time'] = temp['time'].str.replace(' 15:00:59','15:00:59')
    temp['time'] = temp['time'].str.replace(' 9:','09:',regex=True)
    temp['time'] = pd.to_datetime(temp['time']).dt.time
    temp['date'] = pd.to_datetime(temp['date'],dayfirst=True)
    temp['ticker'] = temp['ticker'].str.replace('30MAR23','29MAR23',regex=True)
    temp['Option_Type'] = temp['ticker'].str[-2:]
    temp['Temp'] = temp["ticker"].str.replace(s,"")
    temp['Temp'] = temp['Temp'].str[:-2]
    temp['Length_of_temp'] = temp['Temp'].str.len()
    temp['Strike'] = np.where((temp['Temp'].str.len()==9) | (temp['Temp'].str.len()==11) , 
                              temp['Temp'].str[-4:] , 
                              temp['Temp'].str[-5:])
    temp['Exp_Year'] = np.where((temp['Temp'].str.len()==9) | (temp['Temp'].str.len()==10) ,
                               temp['Temp'].str[:2] ,
                               temp['Temp'].str[5:7])
    temp['Current_Year'] = temp['date'].dt.year
    temp['Current_Year'] = temp['Current_Year'].astype(str).str[-2:]
    temp['Exp_month'] = temp['Temp'].str[2:5]
    temp['Exp_Year'] = temp['Exp_Year'].astype('str')
    temp['MonthYear'] = temp['Exp_month']+temp['Exp_Year']
    temp = pd.merge(temp,exp_date,on='MonthYear')
    temp = temp.drop(['MonthYear','Month','Year','Next_Exp_DT'],axis=1)

    temp['Length_of_temp'] = temp['Length_of_temp'].astype('int64')
    temp_10 = temp[(temp['Length_of_temp']==10) | (temp['Length_of_temp']==9)]

    temp_12 = temp[(temp['Length_of_temp']==12) | (temp['Length_of_temp']==11)]

    temp_12['DateDate'] = temp_12['Temp'].str[:2]
    temp_12['DateDate'] = temp_12['DateDate'].astype('int64')
    temp_12['Exp_DT'] = pd.to_datetime(temp['Exp_DT'],dayfirst=True)
    temp_12['Exp_Day'] = temp_12['Exp_DT'].dt.day
    temp_12 = temp_12[temp_12['Exp_Day']==temp_12['DateDate']]
    temp_12 = temp_12.drop(['DateDate','Exp_Day'],axis=1)

    temp_df = temp_10.append(temp_12,ignore_index=True)

    temp_df['exp_month_number'] = temp_df.apply(lambda row : add(row["Exp_month"]), axis = 1)
    temp_df['New_date'] = temp_df['date']
    temp_df["New_date"] = pd.to_datetime(temp_df["New_date"])
    temp_df["current_month_number"] = temp_df['New_date'].dt.month
    temp_df["difference"] = temp_df['exp_month_number'].astype(int) - temp_df["current_month_number"].astype(int)
    temp_df['Year_difference'] = temp_df['Exp_Year'].astype(int) - temp_df['Current_Year'].astype(int)
    temp_df = temp_df[(temp_df['exp_month_number']==3) | (temp_df['exp_month_number']==6) | (temp_df['exp_month_number']==9) | (temp_df['exp_month_number']==12)]

    temp1 = pd.merge(temp_df, 
                     exp_df, 
                     on ='New_date', 
                     how ='left')

    temp1.drop(temp1.filter(regex="Unname"),axis=1, inplace=True)
    temp1["current_exp_month_number"] = temp1['curr_exp_date'].dt.month
    temp1["Diff_months"] = temp1["current_exp_month_number"] - temp1["current_month_number"]
    temp1["Diff_months"] = temp1["Diff_months"].astype(int) 

    temp1 = temp1[temp1['Exp_DT']>=temp1['curr_exp_date']]                 ## to filter out dates which have wrong ticker

    if os.path.exists(folpath+sym+'-I.csv'):
        os.remove(folpath+sym+'-I.csv')
    if os.path.exists(folpath+sym+'-II.csv'):
        os.remove(folpath+sym+'-II.csv')
    if os.path.exists(folpath+sym+'-III.csv'):
        os.remove(folpath+sym+'-III.csv')
    if os.path.exists(folpath+sym+'-IV.csv'):
        os.remove(folpath+sym+'-IV.csv')


    atemp = temp1[(temp1['Diff_months']==0) & (temp1['Year_difference']==0)]
    agb = atemp.groupby(['difference'])
    unique_a = list(atemp['difference'].unique())

    for i in unique_a:
        temp_df = agb.get_group(i)
        temp_df = temp_df.drop(temp_df.columns[9:],axis=1)
        if i==0 or i==1 or i==2:
            temp_df.to_csv(folpath + sym + '-I.csv', mode='a', header=not os.path.exists(folpath + sym + '-I.csv'), index=False)

        if i==3 or i==4 or i==5:
            temp_df.to_csv(folpath + sym + '-II.csv', mode='a', header=not os.path.exists(folpath + sym + '-II.csv'), index=False)

        if i==6 or i==7 or i==8:
            temp_df.to_csv(folpath + sym + '-III.csv', mode='a', header=not os.path.exists(folpath + sym + '-III.csv'), index=False)

        if i==9 or i==10 or i==11:
            temp_df.to_csv(folpath + sym + '-IV.csv', mode='a', header=not os.path.exists(folpath + sym + '-IV.csv'), index=False)


    btemp = temp1[(temp1['Diff_months']==0) & (temp1['Year_difference']==1)]
    bgb = btemp.groupby(['difference'])
    unique_b = list(btemp['difference'].unique())        

    for i in unique_b:
        temp_df = bgb.get_group(i)
        temp_df = temp_df.drop(temp_df.columns[9:],axis=1)

        if i==-7 or i==-8 or i==-9:
            temp_df.to_csv(folpath + sym + '-II.csv', mode='a', header=not os.path.exists(folpath + sym + '-II.csv'), index=False)

        if i==-4 or i==-5 or i==-6:
            temp_df.to_csv(folpath + sym + '-III.csv', mode='a', header=not os.path.exists(folpath + sym + '-III.csv'), index=False)

        if i==-1 or i==-2 or i==-3:
            temp_df.to_csv(folpath + sym + '-IV.csv', mode='a', header=not os.path.exists(folpath + sym + '-IV.csv'), index=False)



    ctemp = temp1[((temp1['Diff_months']==1) | (temp1['Diff_months']==-11)) & (temp1['Year_difference']==0)]
    cgb = ctemp.groupby(['difference'])
    unique_c = list(ctemp['difference'].unique())

    for i in unique_c:
        temp_df = cgb.get_group(i)
        temp_df = temp_df.drop(temp_df.columns[9:],axis=1)

        if i==1 or i==2 or i==3:
            temp_df.to_csv(folpath + sym + '-I.csv', mode='a', header=not os.path.exists(folpath + sym + '-I.csv'), index=False)

        if i==4 or i==5 or i==6:
            temp_df.to_csv(folpath + sym + '-II.csv', mode='a', header=not os.path.exists(folpath + sym + '-II.csv'), index=False)

        if i==7 or i==8 or i==9:
            temp_df.to_csv(folpath + sym + '-III.csv', mode='a', header=not os.path.exists(folpath + sym + '-III.csv'), index=False)

        if i==10 or i==11:
            temp_df.to_csv(folpath + sym + '-IV.csv', mode='a', header=not os.path.exists(folpath + sym + '-IV.csv'), index=False)


    dtemp = temp1[((temp1['Diff_months']==1) | (temp1['Diff_months']==-11)) & (temp1['Year_difference']==1)]
    dgb = dtemp.groupby(['difference'])
    unique_d = list(dtemp['difference'].unique())

    for i in unique_d:
        temp_df = dgb.get_group(i)
        temp_df = temp_df.drop(temp_df.columns[9:],axis=1)

        if i==-9:
            temp_df.to_csv(folpath + sym + '-I.csv', mode='a', header=not os.path.exists(folpath + sym + '-I.csv'), index=False)

        if i==-6 or i==-7:
            temp_df.to_csv(folpath + sym + '-II.csv', mode='a', header=not os.path.exists(folpath + sym + '-II.csv'), index=False)

        if i==-3 or i==-4:
            temp_df.to_csv(folpath + sym + '-III.csv', mode='a', header=not os.path.exists(folpath + sym + '-III.csv'), index=False)

        if i==0 or i==-1 or i==-2:
            temp_df.to_csv(folpath + sym + '-IV.csv', mode='a', header=not os.path.exists(folpath + sym + '-IV.csv'), index=False)

    for i in range(4):
        if i==0:
            file='I'
        elif i==1:
            file='II'
        elif i==2:
            file='III'
        elif i==3:
            file='IV'
        if os.path.exists(r'C:\users\admin\desktop\Pyspark\Nifty\Quarterly\NIFTY-'+file+'.csv'):
            os.remove(r'C:\users\admin\desktop\Pyspark\Nifty\Quarterly\NIFTY-'+file+'.csv')
        if os.path.exists(r'C:\users\admin\desktop\Pyspark_Contracts\Nifty\Quarterly_Data\NIFTY-'+file+'.csv'):
            ddf = pd.read_csv(r'C:\users\admin\desktop\Pyspark_Contracts\Nifty\Quarterly_Data\NIFTY-'+file+'.csv')
            ddf['Option_Type'] = ddf['ticker'].str[-2:]
            ddf['Strike'] = np.where((ddf['ticker'].str.len()==16) | (ddf['ticker'].str.len()==18) , ddf['ticker'].str[-6:-2] , ddf['ticker'].str[-7:-2])
            ddf['Symbol'] = 'NIFTY' + 'QUARTERLY-' + file + ddf['Strike'].astype(int).astype(str) + ddf['Option_Type']
            ddf['ticker'] = ddf['Symbol']
            ddf = ddf.drop(ddf.columns[9:],axis=1)
            ddf = ddf.rename(columns = {'date':'Date','ticker':'Ticker'})
            ddf = ddf.drop_duplicates()
            ddf.to_csv(r"C:\Users\admin\Desktop\Pyspark\Nifty\Quarterly\NIFTY-"+file+".csv",index=False)
    print("NIFTY QUARTERLY CONTRACTS CREATED")
    
def nifty_halfyearly():
    folpath = r"C:\Users\admin\Desktop\Pyspark_Contracts\Nifty\Half_Yearly_Data\\"
    sym = 'NIFTY'
    s = 'NIFTY'
    def add(stri):
        obj = datetime.strptime(stri, "%b")
        month_number = obj.month
        return month_number
    def get_symbol(tic):
        li = list(filter(None, re.split(r'(\d+)', tic)))
        return li[0]

    exp_date = pd.read_excel(r'C:\users\admin\desktop\Expiry_DT.xlsx')    ## reading the expiry sheet file

    hy_exp_date = pd.read_csv(r"C:\Users\admin\Downloads\half_yearly_expiry.csv",dayfirst=True,parse_dates=['Date','E1'], usecols =['Date','E1'])
    
    ndata = nifty_data()
    temp = ndata.copy()
    temp = temp.loc[:, ~temp.columns.str.contains('^Unnamed')]
    temp = temp.reset_index(drop=True)
    temp['time'] = temp['time'].str.replace(' 15:00:59','15:00:59')
    temp['time'] = temp['time'].str.replace(' 9:','09:',regex=True)
    temp['time'] = pd.to_datetime(temp['time']).dt.time
    temp['date'] = pd.to_datetime(temp['date'],dayfirst=True)
    temp['ticker'] = temp['ticker'].str.replace('30MAR23','29MAR23',regex=True)
    temp['Option_Type'] = temp['ticker'].str[-2:]
    temp['Temp'] = temp["ticker"].str.replace(s,"")
    temp['Temp'] = temp['Temp'].str[:-2]
    temp['Length_of_temp'] = temp['Temp'].str.len()
    temp['Strike'] = np.where((temp['Temp'].str.len()==9) | (temp['Temp'].str.len()==11) , 
                              temp['Temp'].str[-4:] , 
                              temp['Temp'].str[-5:])
    temp['Exp_Year'] = np.where((temp['Temp'].str.len()==9) | (temp['Temp'].str.len()==10) ,
                               temp['Temp'].str[:2] ,
                               temp['Temp'].str[5:7])
    temp['Current_Year'] = temp['date'].dt.year
    temp['Current_Year'] = temp['Current_Year'].astype(str).str[-2:]
    temp['Exp_month'] = temp['Temp'].str[2:5]
    temp['Exp_Year'] = temp['Exp_Year'].astype('str')
    temp['MonthYear'] = temp['Exp_month']+temp['Exp_Year']
    temp = pd.merge(temp,exp_date,on='MonthYear')
    temp = temp.drop(['MonthYear','Month','Year','Next_Exp_DT'],axis=1)
    temp['Length_of_temp'] = temp['Length_of_temp'].astype('int64')
    temp_10 = temp[(temp['Length_of_temp']==10) | (temp['Length_of_temp']==9)]

    temp_12 = temp[(temp['Length_of_temp']==12) | (temp['Length_of_temp']==11)]
    temp_12['DateDate'] = temp_12['Temp'].str[:2]
    temp_12['DateDate'] = temp_12['DateDate'].astype('int64')
    temp_12['Exp_DT'] = pd.to_datetime(temp['Exp_DT'],dayfirst=True)
    temp_12['Exp_Day'] = temp_12['Exp_DT'].dt.day
    temp_12 = temp_12[temp_12['Exp_Day']==temp_12['DateDate']]
    temp_12 = temp_12.drop(['DateDate','Exp_Day'],axis=1)

    temp_df = temp_10.append(temp_12,ignore_index=True)

    temp_df['exp_month_number'] = temp_df.apply(lambda row : add(row["Exp_month"]), axis = 1)
    temp_df['New_date'] = temp_df['date']
    temp_df["New_date"] = pd.to_datetime(temp_df["New_date"])
    temp_df["current_month_number"] = temp_df['New_date'].dt.month
    temp_df["difference"] = temp_df['exp_month_number'].astype(int) - temp_df["current_month_number"].astype(int)
    temp_df['Year_difference'] = temp_df['Exp_Year'].astype(int) - temp_df['Current_Year'].astype(int)
    temp_df = temp_df[(temp_df['exp_month_number']==6) | (temp_df['exp_month_number']==12)]

    hy_exp_date = hy_exp_date.rename({'Date':'New_date'},axis=1)
    temp1 = pd.merge(temp_df, 
                         hy_exp_date, 
                         on ='New_date', 
                         how ='left')

    temp1.drop(temp1.filter(regex="Unname"),axis=1, inplace=True)
    temp1["current_exp_month_number"] = temp1['E1'].dt.month
    temp1["Diff_months"] = temp1["current_exp_month_number"] - temp1["current_month_number"]
    temp1["Diff_months"] = temp1["Diff_months"].astype(int) 

    temp1 = temp1[temp1['Exp_DT']>=temp1['E1']]                 ## to filter out dates which have wrong ticker

    corner_case = temp1[((temp1['current_month_number']==12) & (temp1['current_exp_month_number']==6)) | ((temp1['current_month_number']==6) & (temp1['current_exp_month_number']==12))]
    normal_case = temp1[((temp1['current_month_number']>6) & (temp1['current_month_number']<=12) & (temp1['current_exp_month_number']==12)) | ((temp1['current_month_number']>0) & (temp1['current_month_number']<=6) & (temp1['current_exp_month_number']==6))]
    if(normal_case.shape[0]+corner_case.shape[0]==temp1.shape[0]):
        print("Success")

    if os.path.exists(folpath+sym+'-I.csv'):
        os.remove(folpath+sym+'-I.csv')
    if os.path.exists(folpath+sym+'-II.csv'):
        os.remove(folpath+sym+'-II.csv')
    if os.path.exists(folpath+sym+'-III.csv'):
        os.remove(folpath+sym+'-III.csv')
    if os.path.exists(folpath+sym+'-IV.csv'):
        os.remove(folpath+sym+'-IV.csv')
    if os.path.exists(folpath+sym+'-V.csv'):
        os.remove(folpath+sym+'-V.csv')
    if os.path.exists(folpath+sym+'-VI.csv'):
        os.remove(folpath+sym+'-VI.csv')
    if os.path.exists(folpath+sym+'-VII.csv'):
        os.remove(folpath+sym+'-VII.csv')
    if os.path.exists(folpath+sym+'-VIII.csv'):
        os.remove(folpath+sym+'-VIII.csv')
    if os.path.exists(folpath+sym+'-IX.csv'):
        os.remove(folpath+sym+'-IX.csv')
    if os.path.exists(folpath+sym+'-X.csv'):
        os.remove(folpath+sym+'-X.csv')

    ## NORMAL CASE HY1
    atemp = normal_case[(normal_case['Diff_months']<=5) & (normal_case['Diff_months']>=0) & (normal_case['Year_difference']==0) & ((normal_case['difference']>=0) & (normal_case['difference']<6))]
    agb = atemp.groupby(['Diff_months'])
    unique_a = list(atemp['Diff_months'].unique())
    for i in unique_a:
        temp_df = agb.get_group(i)
        temp_df = temp_df.drop(temp_df.columns[9:],axis=1)
        if i<=5 and i>=0:
            temp_df.to_csv(folpath + sym + '-I.csv', mode='a', header=not os.path.exists(folpath + sym + '-I.csv'), index=False)

    ## NORMAL CASE HY2
    btemp = normal_case[((normal_case['Diff_months']<=5) & (normal_case['Year_difference']==0) & (normal_case['difference']>=6)) | ((normal_case['Diff_months']<=5) & (normal_case['Year_difference']==1) & (normal_case['difference']<=-6))]
    bgb = btemp.groupby(['Diff_months'])
    unique_b = list(btemp['Diff_months'].unique())
    for i in unique_b:
        temp_df = bgb.get_group(i)
        temp_df = temp_df.drop(temp_df.columns[9:],axis=1)
        if i<=6 and i>=0:
            temp_df.to_csv(folpath + sym + '-II.csv', mode='a', header=not os.path.exists(folpath + sym + '-II.csv'), index=False)

    ## NORMAL CASE HY3
    ctemp = normal_case[(normal_case['Diff_months']<=5) & (normal_case['Diff_months']>=0) & (normal_case['Year_difference']==1) & ((normal_case['difference']>=0) & (normal_case['difference']<6))]
    cgb = ctemp.groupby(['Diff_months'])
    unique_c = list(ctemp['Diff_months'].unique())
    for i in unique_c:
        temp_df = cgb.get_group(i)
        temp_df = temp_df.drop(temp_df.columns[9:],axis=1)
        if i<=5 and i>=0:
            temp_df.to_csv(folpath + sym + '-III.csv', mode='a', header=not os.path.exists(folpath + sym + '-III.csv'), index=False)

    ## NORMAL CASE HY4
    dtemp = normal_case[((normal_case['Diff_months']<=5) & (normal_case['Year_difference']==1) & (normal_case['difference']>=6)) | ((normal_case['Diff_months']<=5) & (normal_case['Year_difference']==2) & (normal_case['difference']<=-6))]
    dgb = dtemp.groupby(['Diff_months'])
    unique_d = list(dtemp['Diff_months'].unique())
    for i in unique_d:
        temp_df = dgb.get_group(i)
        temp_df = temp_df.drop(temp_df.columns[9:],axis=1)
        if i<=6 and i>=0:
            temp_df.to_csv(folpath + sym + '-IV.csv', mode='a', header=not os.path.exists(folpath + sym + '-IV.csv'), index=False)

    ## NORMAL CASE HY5
    etemp = normal_case[(normal_case['Diff_months']<=5) & (normal_case['Diff_months']>=0) & (normal_case['Year_difference']==2) & ((normal_case['difference']>=0) & (normal_case['difference']<6))]
    egb = etemp.groupby(['Diff_months'])
    unique_e = list(etemp['Diff_months'].unique())
    for i in unique_e:
        temp_df = egb.get_group(i)
        temp_df = temp_df.drop(temp_df.columns[9:],axis=1)
        if i<=5 and i>=0:
            temp_df.to_csv(folpath + sym + '-V.csv', mode='a', header=not os.path.exists(folpath + sym + '-V.csv'), index=False)

    ## NORMAL CASE HY6
    ftemp = normal_case[((normal_case['Diff_months']<=5) & (normal_case['Year_difference']==2) & (normal_case['difference']>=6)) | ((normal_case['Diff_months']<=5) & (normal_case['Year_difference']==3) & (normal_case['difference']<=-6))]
    fgb = ftemp.groupby(['Diff_months'])
    unique_f = list(ftemp['Diff_months'].unique())
    for i in unique_f:
        temp_df = fgb.get_group(i)
        temp_df = temp_df.drop(temp_df.columns[9:],axis=1)
        if i<=6 and i>=0:
            temp_df.to_csv(folpath + sym + '-VI.csv', mode='a', header=not os.path.exists(folpath + sym + '-VI.csv'), index=False)

    ## NORMAL CASE HY7
    gtemp = normal_case[(normal_case['Diff_months']<=5) & (normal_case['Diff_months']>=0) & (normal_case['Year_difference']==3) & ((normal_case['difference']>=0) & (normal_case['difference']<6))]
    ggb = gtemp.groupby(['Diff_months'])
    unique_g = list(gtemp['Diff_months'].unique())
    for i in unique_g:
        temp_df = ggb.get_group(i)
        temp_df = temp_df.drop(temp_df.columns[9:],axis=1)
        if i<=5 and i>=0:
            temp_df.to_csv(folpath + sym + '-VII.csv', mode='a', header=not os.path.exists(folpath + sym + '-VII.csv'), index=False)

    ## NORMAL CASE HY8
    htemp = normal_case[((normal_case['Diff_months']<=5) & (normal_case['Year_difference']==3) & (normal_case['difference']>=6)) | ((normal_case['Diff_months']<=5) & (normal_case['Year_difference']==4) & (normal_case['difference']<=-6))]
    hgb = htemp.groupby(['Diff_months'])
    unique_h = list(htemp['Diff_months'].unique())
    for i in unique_h:
        temp_df = hgb.get_group(i)
        temp_df = temp_df.drop(temp_df.columns[9:],axis=1)
        if i<=6 and i>=0:
            temp_df.to_csv(folpath + sym + '-VIII.csv', mode='a', header=not os.path.exists(folpath + sym + '-VIII.csv'), index=False)

    ## NORMAL CASE HY9
    itemp = normal_case[(normal_case['Diff_months']<=5) & (normal_case['Diff_months']>=0) & (normal_case['Year_difference']==4) & ((normal_case['difference']>=0) & (normal_case['difference']<6))]
    igb = itemp.groupby(['Diff_months'])
    unique_i = list(itemp['Diff_months'].unique())
    for i in unique_i:
        temp_df = igb.get_group(i)
        temp_df = temp_df.drop(temp_df.columns[9:],axis=1)
        if i<=5 and i>=0:
            temp_df.to_csv(folpath + sym + '-IX.csv', mode='a', header=not os.path.exists(folpath + sym + '-IX.csv'), index=False)

    ## NORMAL CASE HY10
    jtemp = normal_case[((normal_case['Diff_months']<=5) & (normal_case['Year_difference']==4) & (normal_case['difference']>=6)) | ((normal_case['Diff_months']<=5) & (normal_case['Year_difference']==5) & (normal_case['difference']<=-6))]
    jgb = jtemp.groupby(['Diff_months'])
    unique_j = list(jtemp['Diff_months'].unique())
    for i in unique_j:
        temp_df = jgb.get_group(i)
        temp_df = temp_df.drop(temp_df.columns[9:],axis=1)
        if i<=6 and i>=0:
            temp_df.to_csv(folpath + sym + '-X.csv', mode='a', header=not os.path.exists(folpath + sym + '-X.csv'), index=False)

    ## CORNER CASE HY1
    ktemp = corner_case[((corner_case['Diff_months']==6) & (corner_case['Year_difference']==0) & (corner_case['difference']==6)) | ((corner_case['Diff_months']==-6) & (corner_case['Year_difference']==1) & (corner_case['difference']==-6))]
    kgb = ktemp.groupby(['Diff_months'])
    unique_k = list(ktemp['Diff_months'].unique())
    for i in unique_k:
        temp_df = kgb.get_group(i)
        temp_df = temp_df.drop(temp_df.columns[9:],axis=1)
        if i==-6 or i==6:
            temp_df.to_csv(folpath + sym + '-I.csv', mode='a', header=not os.path.exists(folpath + sym + '-I.csv'), index=False)

    ## CORNER CASE HY2
    ltemp = corner_case[((corner_case['Diff_months']==6) & (corner_case['difference']==0) & (corner_case['Year_difference']==1)) | ((corner_case['Diff_months']==-6) & (corner_case['difference']==0) & (corner_case['Year_difference']==1))]
    lgb = ltemp.groupby(['Diff_months'])
    unique_l = list(ltemp['Diff_months'].unique())
    for i in unique_l:
        temp_df = lgb.get_group(i)
        temp_df = temp_df.drop(temp_df.columns[9:],axis=1)
        if i==-6 or i==6:
            temp_df.to_csv(folpath + sym + '-II.csv', mode='a', header=not os.path.exists(folpath + sym + '-II.csv'), index=False)

    ## CORNER CASE HY3
    mtemp = corner_case[((corner_case['Diff_months']==6) & (corner_case['Year_difference']==1) & (corner_case['difference']==6)) | ((corner_case['Diff_months']==-6) & (corner_case['Year_difference']==2) & (corner_case['difference']==-6))]
    mgb = mtemp.groupby(['Diff_months'])
    unique_m = list(mtemp['Diff_months'].unique())
    for i in unique_m:
        temp_df = mgb.get_group(i)
        temp_df = temp_df.drop(temp_df.columns[9:],axis=1)
        if i==-6 or i==6:
            temp_df.to_csv(folpath + sym + '-III.csv', mode='a', header=not os.path.exists(folpath + sym + '-III.csv'), index=False)

    ## CORNER CASE HY4
    ntemp = corner_case[((corner_case['Diff_months']==6) & (corner_case['difference']==0) & (corner_case['Year_difference']==2)) | ((corner_case['Diff_months']==-6) & (corner_case['difference']==0) & (corner_case['Year_difference']==2))]
    ngb = ntemp.groupby(['Diff_months'])
    unique_n = list(ntemp['Diff_months'].unique())
    for i in unique_n:
        temp_df = ngb.get_group(i)
        temp_df = temp_df.drop(temp_df.columns[9:],axis=1)
        if i==-6 or i==6:
            temp_df.to_csv(folpath + sym + '-IV.csv', mode='a', header=not os.path.exists(folpath + sym + '-IV.csv'), index=False)

    ## CORNER CASE HY5
    otemp = corner_case[((corner_case['Diff_months']==6) & (corner_case['Year_difference']==2) & (corner_case['difference']==6)) | ((corner_case['Diff_months']==-6) & (corner_case['Year_difference']==3) & (corner_case['difference']==-6))]
    ogb = otemp.groupby(['Diff_months'])
    unique_o = list(otemp['Diff_months'].unique())
    for i in unique_o:
        temp_df = ogb.get_group(i)
        temp_df = temp_df.drop(temp_df.columns[9:],axis=1)
        if i==-6 or i==6:
            temp_df.to_csv(folpath + sym + '-V.csv', mode='a', header=not os.path.exists(folpath + sym + '-V.csv'), index=False)

    ## CORNER CASE HY6
    ptemp = corner_case[((corner_case['Diff_months']==6) & (corner_case['difference']==0) & (corner_case['Year_difference']==3)) | ((corner_case['Diff_months']==-6) & (corner_case['difference']==0) & (corner_case['Year_difference']==3))]
    pgb = ptemp.groupby(['Diff_months'])
    unique_p = list(ptemp['Diff_months'].unique())
    for i in unique_p:
        temp_df = pgb.get_group(i)
        temp_df = temp_df.drop(temp_df.columns[9:],axis=1)
        if i==-6 or i==6:
            temp_df.to_csv(folpath + sym + '-VI.csv', mode='a', header=not os.path.exists(folpath + sym + '-VI.csv'), index=False)

    ## CORNER CASE HY7
    qtemp = corner_case[((corner_case['Diff_months']==6) & (corner_case['Year_difference']==3) & (corner_case['difference']==6)) | ((corner_case['Diff_months']==-6) & (corner_case['Year_difference']==4) & (corner_case['difference']==-6))]
    qgb = qtemp.groupby(['Diff_months'])
    unique_q = list(qtemp['Diff_months'].unique())
    for i in unique_q:
        temp_df = qgb.get_group(i)
        temp_df = temp_df.drop(temp_df.columns[9:],axis=1)
        if i==-6 or i==6:
            temp_df.to_csv(folpath + sym + '-VII.csv', mode='a', header=not os.path.exists(folpath + sym + '-VII.csv'), index=False)

    ## CORNER CASE HY8
    rtemp = corner_case[((corner_case['Diff_months']==6) & (corner_case['difference']==0) & (corner_case['Year_difference']==4)) | ((corner_case['Diff_months']==-6) & (corner_case['difference']==0) & (corner_case['Year_difference']==4))]
    rgb = rtemp.groupby(['Diff_months'])
    unique_r = list(rtemp['Diff_months'].unique())
    for i in unique_r:
        temp_df = rgb.get_group(i)
        temp_df = temp_df.drop(temp_df.columns[9:],axis=1)
        if i==-6 or i==6:
            temp_df.to_csv(folpath + sym + '-VIII.csv', mode='a', header=not os.path.exists(folpath + sym + '-VIII.csv'), index=False)

    ## CORNER CASE HY9
    stemp = corner_case[((corner_case['Diff_months']==6) & (corner_case['Year_difference']==4) & (corner_case['difference']==6)) | ((corner_case['Diff_months']==-6) & (corner_case['Year_difference']==5) & (corner_case['difference']==-6))]
    sgb = stemp.groupby(['Diff_months'])
    unique_s = list(stemp['Diff_months'].unique())
    for i in unique_s:
        temp_df = sgb.get_group(i)
        temp_df = temp_df.drop(temp_df.columns[9:],axis=1)
        if i==-6 or i==6:
            temp_df.to_csv(folpath + sym + '-IX.csv', mode='a', header=not os.path.exists(folpath + sym + '-IX.csv'), index=False)

    ## CORNER CASE HY10
    ttemp = corner_case[((corner_case['Diff_months']==6) & (corner_case['difference']==0) & (corner_case['Year_difference']==5)) | ((corner_case['Diff_months']==-6) & (corner_case['difference']==0) & (corner_case['Year_difference']==5))]
    tgb = ttemp.groupby(['Diff_months'])
    unique_t = list(ttemp['Diff_months'].unique())
    for i in unique_t:
        temp_df = tgb.get_group(i)
        temp_df = temp_df.drop(temp_df.columns[9:],axis=1)
        if i==-6 or i==6:
            temp_df.to_csv(folpath + sym + '-X.csv', mode='a', header=not os.path.exists(folpath + sym + '-X.csv'), index=False)

    for i in range(10):
        if(i==0):
            file='I'
        elif(i==1):
            file='II'
        elif(i==2):
            file='III'
        elif(i==3):
            file='IV'
        elif(i==4):
            file='V'
        elif(i==5):
            file='VI'
        elif(i==6):
            file='VII'
        elif(i==7):
            file='VIII'
        elif(i==8):
            file='IX'
        else:
            file='X'
        if os.path.exists(r"C:\Users\admin\Desktop\Pyspark\Nifty\Half_Yearly\\Nifty-"+file+'.csv'):
            os.remove(r"C:\Users\admin\Desktop\Pyspark\Nifty\Half_Yearly\\Nifty-"+file+'.csv')
        if os.path.exists(r"C:\Users\admin\Desktop\Pyspark_Contracts\Nifty\Half_Yearly_Data\NIFTY-"+file+".csv"):
            df1 = pd.read_csv(r"C:\Users\admin\Desktop\Pyspark_Contracts\Nifty\Half_Yearly_Data\NIFTY-"+file+".csv")
            df1['Option_Type'] = df1['ticker'].str[-2:]
            df1['Strike'] = np.where((df1['ticker'].str.len()==16) | (df1['ticker'].str.len()==18) , df1['ticker'].str[-6:-2] , df1['ticker'].str[-7:-2])
            df1['Symbol'] = 'NIFTY-' + file + df1['Strike'].astype(int).astype(str) + df1['Option_Type']
            df1['ticker'] = df1['Symbol']
            df1 = df1.drop(df1.columns[9:],axis=1)
            df1.to_csv(r"C:\Users\admin\Desktop\Pyspark\Nifty\Half_Yearly\\Nifty-"+file+'.csv',index=False)
    print("NIFTY HALF YEARLY CONTRACTS CREATED")
    
def nifty_yearly():
    for i in range(5):
        if(i==0):
            file='I'
            file1='I'
            file2='II'
            ## REMOVING YEARLY CONTRACT FILE 
            if os.path.exists(r"C:\Users\admin\Desktop\Pyspark_Contracts\Nifty\Yearly_Data\\Nifty-"+file+'.csv'):
                os.remove(r"C:\Users\admin\Desktop\Pyspark_Contracts\Nifty\Yearly_Data\\Nifty-"+file+'.csv')
            ## CHECKING IF HALFYEARLY FILE EXISTS
            if os.path.exists(r'C:\Users\admin\Desktop\Pyspark_Contracts\Nifty\Half_Yearly_Data\NIFTY-'+file1+'.csv'):
                df1 = pd.read_csv(r'C:\Users\admin\Desktop\Pyspark_Contracts\Nifty\Half_Yearly_Data\NIFTY-'+file1+'.csv')
            else:
                df1 = pd.DataFrame()
            ## CHECKING IF HALFYEARLY FILE EXISTS
            if os.path.exists(r'C:\Users\admin\Desktop\Pyspark_Contracts\Nifty\Half_Yearly_Data\NIFTY-'+file2+'.csv'):
                df2 = pd.read_csv(r'C:\Users\admin\Desktop\Pyspark_Contracts\Nifty\Half_Yearly_Data\NIFTY-'+file2+'.csv')
            else:
                df2 = pd.DataFrame()
        elif(i==1):
            file='II'
            file1='III'
            file2='IV'
            ## REMOVING YEARLY CONTRACT FILE 
            if os.path.exists(r"C:\Users\admin\Desktop\Pyspark_Contracts\Nifty\Yearly_Data\\Nifty-"+file+'.csv'):
                os.remove(r"C:\Users\admin\Desktop\Pyspark_Contracts\Nifty\Yearly_Data\\Nifty-"+file+'.csv')
            ## CHECKING IF HALFYEARLY FILE EXISTS
            if os.path.exists(r'C:\Users\admin\Desktop\Pyspark_Contracts\Nifty\Half_Yearly_Data\NIFTY-'+file1+'.csv'):
                df1 = pd.read_csv(r'C:\Users\admin\Desktop\Pyspark_Contracts\Nifty\Half_Yearly_Data\NIFTY-'+file1+'.csv')
            else:
                df1 = pd.DataFrame()
            ## CHECKING IF HALFYEARLY FILE EXISTS
            if os.path.exists(r'C:\Users\admin\Desktop\Pyspark_Contracts\Nifty\Half_Yearly_Data\NIFTY-'+file2+'.csv'):
                df2 = pd.read_csv(r'C:\Users\admin\Desktop\Pyspark_Contracts\Nifty\Half_Yearly_Data\NIFTY-'+file2+'.csv')
            else:
                df2 = pd.DataFrame()
        elif(i==2):
            file='III'
            file1='V'
            file2='VI'
            ## REMOVING YEARLY CONTRACT FILE 
            if os.path.exists(r"C:\Users\admin\Desktop\Pyspark_Contracts\Nifty\Yearly_Data\\Nifty-"+file+'.csv'):
                os.remove(r"C:\Users\admin\Desktop\Pyspark_Contracts\Nifty\Yearly_Data\\Nifty-"+file+'.csv')
            if os.path.exists(r'C:\Users\admin\Desktop\Pyspark_Contracts\Nifty\Half_Yearly_Data\NIFTY-'+file1+'.csv'):
                df1 = pd.read_csv(r'C:\Users\admin\Desktop\Pyspark_Contracts\Nifty\Half_Yearly_Data\NIFTY-'+file1+'.csv')
            else:
                df1 = pd.DataFrame()
            if os.path.exists(r'C:\Users\admin\Desktop\Pyspark_Contracts\Nifty\Half_Yearly_Data\NIFTY-'+file2+'.csv'):
                df2 = pd.read_csv(r'C:\Users\admin\Desktop\Pyspark_Contracts\Nifty\Half_Yearly_Data\NIFTY-'+file2+'.csv')
            else:
                df2 = pd.DataFrame()
        elif(i==3):
            file='IV'
            file1='VII'
            file2='VIII'
            ## REMOVING YEARLY CONTRACT FILE 
            if os.path.exists(r"C:\Users\admin\Desktop\Pyspark_Contracts\Nifty\Yearly_Data\\Nifty-"+file+'.csv'):
                os.remove(r"C:\Users\admin\Desktop\Pyspark_Contracts\Nifty\Yearly_Data\\Nifty-"+file+'.csv')
            ## CHECKING IF HALFYEARLY FILE EXISTS
            if os.path.exists(r'C:\Users\admin\Desktop\Pyspark_Contracts\Nifty\Half_Yearly_Data\NIFTY-'+file1+'.csv'):
                df1 = pd.read_csv(r'C:\Users\admin\Desktop\Pyspark_Contracts\Nifty\Half_Yearly_Data\NIFTY-'+file1+'.csv')
            else:
                df1 = pd.DataFrame()
            ## CHECKING IF HALFYEARLY FILE EXISTS
            if os.path.exists(r'C:\Users\admin\Desktop\Pyspark_Contracts\Nifty\Half_Yearly_Data\NIFTY-'+file2+'.csv'):
                df2 = pd.read_csv(r'C:\Users\admin\Desktop\Pyspark_Contracts\Nifty\Half_Yearly_Data\NIFTY-'+file2+'.csv')
            else:
                df2 = pd.DataFrame()
        elif(i==4):
            file='V'
            file1='IX'
            file2='X'
            ## REMOVING YEARLY CONTRACT FILE 
            if os.path.exists(r"C:\Users\admin\Desktop\Pyspark_Contracts\Nifty\Yearly_Data\\Nifty-"+file+'.csv'):
                os.remove(r"C:\Users\admin\Desktop\Pyspark_Contracts\Nifty\Yearly_Data\\Nifty-"+file+'.csv')
            ## CHECKING IF HALFYEARLY FILE EXISTS
            if os.path.exists(r'C:\Users\admin\Desktop\Pyspark_Contracts\Nifty\Half_Yearly_Data\NIFTY-'+file1+'.csv'):
                df1 = pd.read_csv(r'C:\Users\admin\Desktop\Pyspark_Contracts\Nifty\Half_Yearly_Data\NIFTY-'+file1+'.csv')
            else:
                df1 = pd.DataFrame()
            ## CHECKING IF HALFYEARLY FILE EXISTS
            if os.path.exists(r'C:\Users\admin\Desktop\Pyspark_Contracts\Nifty\Half_Yearly_Data\NIFTY-'+file2+'.csv'):
                df2 = pd.read_csv(r'C:\Users\admin\Desktop\Pyspark_Contracts\Nifty\Half_Yearly_Data\NIFTY-'+file2+'.csv')
            else:
                df2 = pd.DataFrame()
        
        if df1.empty and df2.empty:
            print("No contracts for Half yearly",file1,file2)
        else:
            print(i,"YEARLY",file,"HALFYEARLY",file1,file2)
            if not df1.empty:
                df1 = df1.sort_values(by='date')
                df1['Month'] = df1['ticker'].str[7:10]
                df1 = df1[df1['Month']=='DEC']
            if not df2.empty:
                df2 = df2.sort_values(by='date')
                df2['Month'] = df2['ticker'].str[7:10]
                df2 = df2[df2['Month']=='DEC']
            final_df = df1.append(df2,ignore_index=True)
            if not final_df.empty:
                final_df = final_df.drop_duplicates()
                final_df = final_df.drop(final_df.columns[9:],axis=1)
                final_df.to_csv(r"C:\Users\admin\Desktop\Pyspark_Contracts\Nifty\Yearly_Data\\Nifty-"+file+'.csv',index=False)

    for i in range(5):
        if(i==0):
            file='I'
        if(i==1):
            file='II'
        if(i==2):
            file='III'
        if(i==3):
            file='IV'
        if(i==4):
            file='V'
        if os.path.exists(r"C:\Users\admin\Desktop\Pyspark\Nifty\Yearly\\Nifty-"+file+".csv"):
            os.remove(r"C:\Users\admin\Desktop\Pyspark\Nifty\Yearly\\Nifty-"+file+".csv")
        if os.path.exists(r"C:\Users\admin\Desktop\Pyspark_Contracts\Nifty\Yearly_Data\\Nifty-"+file+'.csv'):
            df = pd.read_csv(r"C:\Users\admin\Desktop\Pyspark_Contracts\Nifty\Yearly_Data\\Nifty-"+file+'.csv')
            df['Option_Type'] = df['ticker'].str[-2:]
            df['Strike'] = np.where((df['ticker'].str.len()==16) | (df['ticker'].str.len()==18) , df['ticker'].str[-6:-2] , df['ticker'].str[-7:-2])
            df['Symbol'] = 'NIFTY-' + file + df['Strike'].astype(int).astype(str) + df['Option_Type']
            df['ticker'] = df['Symbol']
            df = df.drop(df.columns[9:],axis=1)
            df.to_csv(r"C:\Users\admin\Desktop\Pyspark\Nifty\Yearly\\Nifty-"+file+".csv",index=False)
    print("YEARLY CONTRACTS GENERATED")

def finnifty_data():
    spark = SparkSession.builder.config("spark.jars", "C:\\Users\\admin\\Downloads\\postgresql-42.5.0.jar") \
    .master("local").appName("PySpark_Postgres_test").getOrCreate()
    #spark.sparkContext.setLogLevel("WARN")
    df = spark.read.format("jdbc").option("url", "jdbc:postgresql://swandatabase.cfehmk2wtejq.ap-south-1.rds.amazonaws.com/RawDataBase").option("user","postgres").option("password","swancap123")\
        .option("driver", "org.postgresql.Driver").option("dbtable", tablename)\
        .option("user", "postgres").option("password", "swancap123").load()
    ## GETTING ONLY TIME IN TIME COLUMN
    q = df.withColumn('time',date_format('time', 'HH:mm:ss'))
    ## FILTERING FINNIFTY DATA
    fndata = q.filter(q.ticker.contains('FINNIFTY') & ((q.ticker.endswith('E.NFO'))| (q.ticker.endswith('E'))))
    ## REPLACING .NFO IN ticker
    fndata = fndata.withColumn('ticker',regexp_replace('ticker','.NFO',''))
    fndata = fndata.drop(col('ticker_check'))
    ## CONVERTING PYSPARK DATAFRAME TO PANDAS DATAFRAME
    fndata=fndata.toPandas()
    return fndata

def finnifty_monthly():
    exp_df = pd.read_excel(r"C:\Users\admin\Desktop\Expiry_Dates\Finnifty.xlsx",sheet_name='Monthly')
    exp_df.rename({'curr_date': 'New_date'}, axis=1, inplace=True)
    exp_date = pd.read_excel(r"C:\Users\admin\Desktop\Expiry_Dates\Finnifty_Monthly_Expiry.xlsx")

    folpath = r"C:\Users\admin\Desktop\Pyspark_Contracts\FinNifty\Monthly_Data\\"
    finalpath = r"C:\Users\admin\Desktop\Pyspark\FinNifty\Monthly\\"
    s = 'FINNIFTY'
    def add(stri):
        obj = datetime.strptime(stri, "%b")
        month_number = obj.month
        return month_number
    sym = 'FINNIFTY'
    fndata = finnifty_data()
    dff = fndata.copy()
    dff = dff[dff['ticker'].str.contains('FINNIFTY')]
    dff = dff[(dff['ticker'].str.endswith('E')) | (dff['ticker'].str.endswith('E.NFO'))]
    dff = dff.loc[:, ~dff.columns.str.contains('^Unnamed')]
    dff['time'] = pd.to_datetime(dff['time']).dt.time
    dff['date'] = pd.to_datetime(dff['date'],dayfirst=True)
    dff['ticker'] = dff['ticker'].str.replace('30MAR23','29MAR23',regex=True)

    dff['Option_Type'] = dff['ticker'].str[-2:]
    dff['Temp'] = dff["ticker"].str.replace(s,"")
    dff['Temp'] = dff['Temp'].str[:-2]
    dff['Length_of_Temp'] = dff['Temp'].str.len()
    dff['Strike'] = np.where((dff['Temp'].str.len()==9) | (dff['Temp'].str.len()==11) , 
                              dff['Temp'].str[-4:] , 
                              dff['Temp'].str[-5:])
    dff['Exp_Year'] = np.where((dff['Temp'].str.len()==9) | (dff['Temp'].str.len()==10) ,
                               dff['Temp'].str[:2] ,
                               dff['Temp'].str[5:7])
    dff['Exp_month'] = dff['Temp'].str[2:5]

    dff['Exp_Year'] = dff['Exp_Year'].astype('str')
    dff['MonthYear'] = dff['Exp_month']+dff['Exp_Year']
    dff = pd.merge(dff,exp_date,on='MonthYear')
    dff = dff.drop(['MonthYear','Month','Year'],axis=1)
    dff = dff.rename(columns={'Exp_DT':'Monthly_Expiry'})

    dff['Length_of_Temp'] = dff['Length_of_Temp'].astype('int64')
    temp_10 = dff[(dff['Length_of_Temp']==10) | (dff['Length_of_Temp']==9)]

    temp_12 = dff[(dff['Length_of_Temp']==12) | (dff['Length_of_Temp']==11)]
    temp_12['DateDate'] = temp_12['Temp'].str[:2]
    temp_12['DateDate'] = temp_12['DateDate'].astype('int64')
    temp_12['Exp_DT'] = pd.to_datetime(temp_12['Monthly_Expiry'],dayfirst=True)
    temp_12['Exp_Day'] = temp_12['Exp_DT'].dt.day
    temp_12 = temp_12[temp_12['Exp_Day']==temp_12['DateDate']]
    temp_12 = temp_12.drop(['DateDate','Exp_Day'],axis=1)

    temp_df = temp_10.append(temp_12,ignore_index=True)
    temp_df['exp_month_number'] = temp_df.apply(lambda row : add(row["Exp_month"]), axis = 1)
    temp_df['New_date'] = temp_df['date']
    temp_df["New_date"] = pd.to_datetime(temp_df["New_date"])
    
    temp_df["current_month_number"] = temp_df['New_date'].dt.month
    temp_df["difference"] = temp_df['exp_month_number'].astype(int) - temp_df["current_month_number"].astype(int)

    temp_df1 = pd.merge(temp_df, 
                     exp_df, 
                     on ='New_date', 
                     how ='left')
    temp_df1.drop(temp_df1.filter(regex="Unname"),axis=1, inplace=True)
    temp_df1["current_exp_month_number"] = temp_df1['curr_exp_date'].dt.month
    temp_df1["Diff_months"] = temp_df1["current_exp_month_number"] - temp_df1["current_month_number"]
    temp_df1["Diff_months"] = temp_df1["Diff_months"].astype(int) 
    temp_df1['Current_Year'] = temp_df1['New_date'].dt.year.astype(str).str[-2:]

    bdf = temp_df1[temp_df1["Diff_months"] == 0]
    adf = temp_df1[(temp_df1["Diff_months"] == 1) | (temp_df1["Diff_months"] == -11)]
    agb = adf.groupby(["difference"])
    unique_val_list_a = list(adf["difference"].unique())

    bgb = bdf.groupby(["difference"])
    unique_val_list_b = list(bdf["difference"].unique())

    ## TO AVOID OVERWRITING, REMOVING FILE
    for i in range(3):
        if(i==0):
            file='-I'
        if(i==1):
            file='-II'
        if(i==2):
            file='-III'
        if(os.path.exists(folpath+sym+file+".csv")):
            os.remove(folpath+sym+file+".csv")

    for i in unique_val_list_b:
        temp_df_new = bgb.get_group(i)
        temp_df_new = temp_df_new.drop(temp_df_new.columns[9:],axis=1)
        if i == 0:
            temp_df_new.to_csv(folpath + sym + '-I.csv', mode = 'a', header = not os.path.exists(folpath + sym + '-I.csv'), index=False)

        if i == 1 or i == -11:
            temp_df_new.to_csv(folpath + sym + '-II.csv', mode = 'a', header = not os.path.exists(folpath + sym + '-II.csv'), index=False)

        if i == 2 or i == -10:
            temp_df_new.to_csv(folpath + sym + '-III.csv', mode = 'a', header = not os.path.exists(folpath + sym + '-III.csv'), index=False)

    for i in unique_val_list_a:
        temp_df_new = agb.get_group(i)
        temp_df_new = temp_df_new.drop(temp_df_new.columns[9:],axis=1)
        if i == 1 or i == -11:
            temp_df_new.to_csv(folpath + sym + '-I.csv', mode = 'a', header = not os.path.exists(folpath + sym + '-I.csv'), index=False)

        if i == 2 or i == -10:
            temp_df_new.to_csv(folpath + sym + '-II.csv', mode = 'a', header = not os.path.exists(folpath + sym + '-II.csv'), index=False)

        if i == 3 or i == -9:
            temp_df_new.to_csv(folpath + sym + '-III.csv', mode = 'a', header = not os.path.exists(folpath + sym + '-III.csv'), index=False)

    for i in range(3):
        if(i==0):
            file='-I'
        if(i==1):
            file='-II'
        if(i==2):
            file='-III'

        if os.path.exists(finalpath+sym+file+".csv"):
            os.remove(finalpath+sym+file+".csv")
        if os.path.exists(folpath+sym+file+".csv"):
            df = pd.read_csv(folpath+sym+file+".csv")
            df['Option_Type'] = df['ticker'].str[-2:]
            df['Strike'] = np.where((df['ticker'].str.len()==19) | (df['ticker'].str.len()==21) , df['ticker'].str[-6:-2] , df['ticker'].str[-7:-2])
            df['Symbol'] = 'FINNIFTYMONTHLY' + file + df['Strike'].astype(int).astype(str) + df['Option_Type']
            df['ticker'] = df['Symbol']
            df = df.drop(df.columns[9:],axis=1)
            df = df.sort_values(by='date')
            print(file,df.shape[0])
            df = df.drop_duplicates()
            print(file,df.shape[0])
            df.to_csv(finalpath+sym+file+".csv",index=False)
    print("FINNIFTY MONTHLY CONTRACTS GENERATED")
    
def finnifty_weekly():
    exp_df1 = pd.read_excel(r"C:\Users\admin\Desktop\Expiry_Dates\Finnifty.xlsx",sheet_name='Weekly',parse_dates=['date'],usecols= ['date','Week_number'])
    exp_df2 = pd.read_excel(r"C:\Users\admin\Desktop\Expiry_Dates\Finnifty.xlsx",sheet_name='Weekly',parse_dates=['Weekly_Expiry_Date'],usecols= ['Weekly_Expiry_Date', 'Expiry_Week_number'])
    exp_date = pd.read_excel(r"C:\Users\admin\Desktop\Expiry_Dates\Finnifty_Monthly_Expiry.xlsx")

    folpath = r"C:\Users\admin\Desktop\Pyspark_Contracts\FinNifty\Weekly_Data\\"
    finalpath = r"C:\Users\admin\Desktop\Pyspark\FinNifty\Weekly\\"
    s = 'FINNIFTY'
    def add(stri):
        obj = datetime.strptime(stri, "%b")
        month_number = obj.month
        return month_number
    sym = 'FINNIFTY'
    fndata = finnifty_data()
    dff = fndata.copy()
    dff = dff[dff['ticker'].str.contains('FINNIFTY')]
    dff = dff[(dff['ticker'].str.endswith('E')) | (dff['ticker'].str.endswith('E.NFO'))]
    dff = dff.loc[:, ~dff.columns.str.contains('^Unnamed')]
    dff['time'] = pd.to_datetime(dff['time']).dt.time
    dff['date'] = pd.to_datetime(dff['date'],dayfirst=True)
    dff['ticker'] = dff['ticker'].str.replace('30MAR23','29MAR23',regex=True)

    dff['Option_Type'] = dff['ticker'].str[-2:]
    dff['Temp'] = dff["ticker"].str.replace(s,"")
    dff['Temp'] = dff['Temp'].str[:-2]
    dff['Length_of_Temp'] = dff['Temp'].str.len()
    dff['Strike'] = np.where((dff['Temp'].str.len()==9) | (dff['Temp'].str.len()==11) , 
                              dff['Temp'].str[-4:] , 
                              dff['Temp'].str[-5:])
    dff['Exp_Year'] = np.where((dff['Temp'].str.len()==9) | (dff['Temp'].str.len()==10) ,
                               dff['Temp'].str[:2] ,
                               dff['Temp'].str[5:7])
    dff['Exp_month'] = dff['Temp'].str[2:5]
    dff['EXPIRY_DT'] = dff['Temp'].str[:7]
    dff['EXPIRY_DT'] = pd.to_datetime(dff['EXPIRY_DT'],dayfirst=True)

    dff['Exp_Year'] = dff['Exp_Year'].astype('str')
    dff['MonthYear'] = dff['Exp_month']+dff['Exp_Year']
    dff = pd.merge(dff,exp_date,on='MonthYear')
    dff = dff.drop(['MonthYear','Month','Year'],axis=1)
    dff = dff.rename(columns={'Exp_DT':'Monthly_Expiry'})

    ## GETTING EXPIRY DATES FOR MONTHLY CONTRACTS
    dff['expiry_date'] = np.where(dff['Length_of_Temp']>=11,dff['EXPIRY_DT'],dff['Monthly_Expiry'])
    dff = dff.drop(['EXPIRY_DT',"Monthly_Expiry"],axis=1)
    ## GETTING WEEK NUMBER OF CURRENT DATE
    dff = dff.rename(columns={'Date':'date'})
    dff = pd.merge(dff,exp_df1,on='date',how='left')

    ## GETTING WEEK NUMBER OF EXPIRY DATES
    exp_df2['Weekly_Expiry_Date'] = pd.to_datetime(exp_df2['Weekly_Expiry_Date'],dayfirst=True)
    exp_df2 = exp_df2.dropna()
    exp_df2 = exp_df2.rename(columns = {'Weekly_Expiry_Date':'expiry_date'})
    temp_df = pd.merge(dff, exp_df2, on = 'expiry_date', how = 'left')
    temp_df = temp_df.drop_duplicates()
    temp_df['week_diff'] = temp_df['Expiry_Week_number'] - temp_df['Week_number']
    final_df = temp_df.copy()
    final_df["week_diff"] = final_df['week_diff'].replace(np.nan,10000)
    
    agb = final_df.groupby(["week_diff"])
    unique_val_list_a = list(final_df["week_diff"].unique())
    unique_val_list_a = sorted([a for a in unique_val_list_a if a>=0])[0:14]
    print(unique_val_list_a)
    for i in range(12):
        if(i==0):
            file='-I'
        if(i==1):
            file='-II'
        if(i==2):
            file='-III'
        if(i==3):
            file='-IV'
        if(i==4):
            file='-V'
        if(i==5):
            file='-VI'
        if(i==6):
            file='-VII'
        if(i==7):
            file='-VIII'
        if(i==8):
            file='-IX'
        if(i==9):
            file='-X'
        if(i==10):
            file='-XI'
        if(i==11):
            file='-XII'
            
        if(os.path.exists(folpath+sym+file+".csv")):
            os.remove(folpath+sym+file+".csv")

    for i in sorted(unique_val_list_a):
        temp_df = agb.get_group(i)
        temp_df = temp_df.drop(temp_df.columns[9:],axis=1)
        if i == 0:
            temp_df.to_csv(folpath + s + '-I.csv', mode = 'a', header = not os.path.exists(folpath + s + '-I.csv'), index=False)

        if i == 1:
            temp_df.to_csv(folpath + s + '-II.csv', mode = 'a', header = not os.path.exists(folpath + s + '-II.csv'), index=False)

        if i == 2:
            temp_df.to_csv(folpath + s + '-III.csv', mode = 'a', header = not os.path.exists(folpath + s + '-III.csv'), index=False)

        if i == 3:
            temp_df.to_csv(folpath + s + '-IV.csv', mode = 'a', header = not os.path.exists(folpath + s + '-IV.csv'), index=False)

        if i == 4:
            temp_df.to_csv(folpath + s + '-V.csv', mode = 'a', header = not os.path.exists(folpath + s + '-V.csv'), index=False)

        if i == 5:
            temp_df.to_csv(folpath + s + '-VI.csv', mode = 'a', header = not os.path.exists(folpath + s + '-VI.csv'), index=False)

        if i == 6:
            temp_df.to_csv(folpath + s + '-VII.csv', mode = 'a', header = not os.path.exists(folpath + s + '-VII.csv'), index=False)

        if i == 7:
            temp_df.to_csv(folpath + s + '-VIII.csv', mode = 'a', header = not os.path.exists(folpath + s + '-VIII.csv'), index=False)

        if i == 8:
            temp_df.to_csv(folpath + s + '-IX.csv', mode = 'a', header = not os.path.exists(folpath + s + '-IX.csv'), index=False)

        if i == 9:
            temp_df.to_csv(folpath + s + '-X.csv', mode = 'a', header = not os.path.exists(folpath + s + '-X.csv'), index=False)

        if i == 10:
            temp_df.to_csv(folpath + s + '-XI.csv', mode = 'a', header = not os.path.exists(folpath + s + '-XI.csv'), index=False)

        if i == 11:
            temp_df.to_csv(folpath + s + '-XII.csv', mode = 'a', header = not os.path.exists(folpath + s + '-XII.csv'), index=False)

        if i == 12:
            temp_df.to_csv(folpath + s + '-XIII.csv', mode = 'a', header = not os.path.exists(folpath + s + '-XIII.csv'), index=False)

        if i == 13:
            temp_df.to_csv(folpath + s + '-XIV.csv', mode = 'a', header = not os.path.exists(folpath + s + '-XIV.csv'), index=False)

    for i in range(12):
        if(i==0):
            file='-I'
        if(i==1):
            file='-II'
        if(i==2):
            file='-III'
        if(i==3):
            file='-IV'
        if(i==4):
            file='-V'
        if(i==5):
            file='-VI'
        if(i==6):
            file='-VII'
        if(i==7):
            file='-VIII'
        if(i==8):
            file='-IX'
        if(i==9):
            file='-X'
        if(i==10):
            file='-XI'
        if(i==11):
            file='-XII'
        if(i==12):
            file='-XIII'
        if(i==13):
            file='-XIV'
        if os.path.exists(finalpath+sym+file+".csv"):
            os.remove(finalpath+sym+file+".csv")
        if(os.path.exists(folpath+sym+file+".csv")):
            print(file)
            df = pd.read_csv(folpath+sym+str(file)+".csv")
            df['Option_Type'] = df['ticker'].str[-2:]
            df['Strike'] = np.where((df['ticker'].str.len()==19) | (df['ticker'].str.len()==21) , df['ticker'].str[-6:-2] , df['ticker'].str[-7:-2])
            df['Symbol'] = 'FINNIFTYWEEKLY' + file + df['Strike'].astype(int).astype(str) + df['Option_Type']
            df['ticker'] = df['Symbol']
            df = df.drop(df.columns[9:],axis=1)
            df = df.sort_values(by='date')
            print(df.shape[0])
            df = df.drop_duplicates()
            print(df.shape[0])
            df.to_csv(finalpath+sym+str(file)+".csv",index=False)
        else:
            print(file,"not exists")
    print("FINNIFTY WEEKLY CONTRACTS CREATED")

    
print("BANKNIFTY CONTRACTS BEING CREATED")
banknifty_monthly()
banknifty_weekly()
banknifty_quarterly()
banknifty_halfyearly()
banknifty_yearly()
print("\nBANKNIFTY CONTRACTS CREATED")

print("\n\nNIFTY CONTRACTS BEING CREATED")
nifty_monthly()
nifty_weekly()
nifty_quarterly()
nifty_halfyearly()
nifty_yearly()
print("\nNIFTY CONTRACTS CREATED")

print("\n\nFINNIFTY CONTRACTS BEING CREATED")
finnifty_monthly()
finnifty_weekly()
print("\nFINNIFTY CONTRACTS CREATED")

et = time.time()
print("ELAPSED TIME",et-st)

Enter a date in YYYY-MM-DD format 2023-05-10
r10052023
BANKNIFTY CONTRACTS BEING CREATED
Sanity Check Success
BANKNIFTY MONTHLY CONTRACTS CREATED
[0.0, 1.0, 2.0, 3.0, 4.0, 7.0, 11.0, 10000.0]
BANKNIFTY WEEKLY CONTRACTS CREATED
BANKNIFTY QUARTERLY CONTRACTS CREATED
BANKNIFTY HALF-YEARLY-I CREATED
BANKNIFTY-QUARTERLY-III not found
BANKNIFTY-QUARTERLY-IV not found
BANKNIFTY HALFYEARLY CONTRACTS CREATED
0 I
1 II
2 III
BANKNIFTY QUARTERLY-III does not exist
3 III
BANKNIFTY QUARTERLY-III does not exist
Dataframe is empty!!!
BANKNIFTY YEARLY CONTRACTS CREATED

BANKNIFTY CONTRACTS CREATED


NIFTY CONTRACTS BEING CREATED
Sanity Check Success
NIFTY MONTHLY CONTRACTS CREATED
[0.0, 1.0, 2.0, 3.0, 4.0, 7.0, 11.0, 10000.0]
NIFTY WEEKLY CONTRACTS CREATED
NIFTY QUARTERLY CONTRACTS CREATED
Success
NIFTY HALF YEARLY CONTRACTS CREATED
0 YEARLY I HALFYEARLY I II
1 YEARLY II HALFYEARLY III IV
2 YEARLY III HALFYEARLY V VI
3 YEARLY IV HALFYEARLY VII VIII
No contracts for Half yearly IX X
YEARLY CONTRACTS GEN